# Deep Research Sustainability Screening Agent
This workdlow uses a hybrid Parallelization and Evaluator–Optimizer workflow pattern to perform deep research on a company’s adoption of innovative sustainable technologies. It enables investment teams to assess ESG leadership using dynamic, internet-scale data.


The following code was adapted from [Google Gemini's Langgraph Research Agent Repo](https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart)

## Configuration

In [6]:
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableConfig
from typing import Any, Optional
import os

class Configuration(BaseModel):
    """The configuration for the agent."""

    # Must be a google model
    query_generator_model: str = Field(
        default="gemini-2.0-flash",
        metadata={
            "description": "The name of the language model to use for the agent's query generation."
        },
    )

    reflection_model: str = Field(
        default="gpt-5-mini-2025-08-07",
        metadata={
            "description": "The name of the language model to use for the agent's reflection."
        },
    )

    reasoning_model: str = Field(
        default='gpt-5-mini-2025-08-07',
        metadata={"description": "The reasoning model for complex analysis."},
    )

    number_of_initial_queries: int = Field(
        default=3,
        metadata={"description": "The number of initial search queries to generate."},
    )

    max_research_loops: int = Field(
        default=2,
        metadata={"description": "The maximum number of research loops to perform."},
    )
    
    @classmethod
    def from_runnable_config(
        cls, config: Optional[RunnableConfig] = None
    ) -> "Configuration":
        """Create a Configuration instance from a RunnableConfig."""
        configurable = (
            config["configurable"] if config and "configurable" in config else {}
        )

        # Get raw values from environment or config
        raw_values: dict[str, Any] = {
            name: os.environ.get(name.upper(), configurable.get(name))
            for name in cls.model_fields.keys()
        }

        # Filter out None values
        values = {k: v for k, v in raw_values.items() if v is not None}

        return cls(**values)


## State

In [7]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
import operator

class OverallState(TypedDict):
    messages: Annotated[list, add_messages]
    sources_gathered: Annotated[list, operator.add]
    sources_used:Annotated[list, operator.add]
    web_research_result: Annotated[list, operator.add]
    search_query: Annotated[list, operator.add]
    initial_search_query_count: int
    max_research_loops: int
    research_loop_count: int
    

class Query(TypedDict):
    query: str
    rationale: str

class QueryGenerationState(TypedDict):
    query_list: list[Query]

class WebSearchState(TypedDict):
    search_query: str
    id: str

class ReflectionState(TypedDict):
    is_sufficient: bool
    knowledge_gap: str
    follow_up_queries: Annotated[list, operator.add]
    research_loop_count: int
    number_of_ran_queries: int

## Utils

In [8]:
from typing import Any, Dict, List
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage
from datetime import datetime

def get_current_date():
    return datetime.now().strftime("%B %d, %Y")

def get_research_topic(messages: List[AnyMessage]) -> str:
    """
    Get the research topic from the messages.
    """
    # check if request has a history and combine the messages into a single string
    if len(messages) == 1:
        research_topic = messages[-1].content
    else:
        research_topic = ""
        for message in messages:
            if isinstance(message, HumanMessage):
                research_topic += f"User: {message.content}\n"
            elif isinstance(message, AIMessage):
                research_topic += f"Assistant: {message.content}\n"
    return research_topic


def resolve_urls(urls_to_resolve: List[Any], id: int) -> Dict[str, str]:
    """
    Create a map of the vertex ai search urls (very long) to a short url with a unique id for each url.
    Ensures each original URL gets a consistent shortened form while maintaining uniqueness.
    """
    prefix = f"https://vertexaisearch.cloud.google.com/id/"
    urls = [site.web.uri for site in urls_to_resolve]

    # Create a dictionary that maps each unique URL to its first occurrence index
    resolved_map = {}
    for idx, url in enumerate(urls):
        if url not in resolved_map:
            resolved_map[url] = f"{prefix}{id}-{idx}"

    return resolved_map


def insert_citation_markers(text, citations_list):
    """
    Inserts citation markers into a text string based on start and end indices.

    Args:
        text (str): The original text string.
        citations_list (list): A list of dictionaries, where each dictionary
                               contains 'start_index', 'end_index', and
                               'segment_string' (the marker to insert).
                               Indices are assumed to be for the original text.

    Returns:
        str: The text with citation markers inserted.
    """
    # Sort citations by end_index in descending order.
    # If end_index is the same, secondary sort by start_index descending.
    # This ensures that insertions at the end of the string don't affect
    # the indices of earlier parts of the string that still need to be processed.
    sorted_citations = sorted(
        citations_list, key=lambda c: (c["end_index"], c["start_index"]), reverse=True
    )

    modified_text = text
    for citation_info in sorted_citations:
        # These indices refer to positions in the *original* text,
        # but since we iterate from the end, they remain valid for insertion
        # relative to the parts of the string already processed.
        end_idx = citation_info["end_index"]
        marker_to_insert = ""
        for segment in citation_info["segments"]:
            marker_to_insert += f" [{segment['label']}]({segment['short_url']})"
        # Insert the citation marker at the original end_idx position
        modified_text = (
            modified_text[:end_idx] + marker_to_insert + modified_text[end_idx:]
        )

    return modified_text


def get_citations(response, resolved_urls_map):
    """
    Extracts and formats citation information from a Gemini model's response.

    This function processes the grounding metadata provided in the response to
    construct a list of citation objects. Each citation object includes the
    start and end indices of the text segment it refers to, and a string
    containing formatted markdown links to the supporting web chunks.

    Args:
        response: The response object from the Gemini model, expected to have
                  a structure including `candidates[0].grounding_metadata`.
                  It also relies on a `resolved_map` being available in its
                  scope to map chunk URIs to resolved URLs.

    Returns:
        list: A list of dictionaries, where each dictionary represents a citation
              and has the following keys:
              - "start_index" (int): The starting character index of the cited
                                     segment in the original text. Defaults to 0
                                     if not specified.
              - "end_index" (int): The character index immediately after the
                                   end of the cited segment (exclusive).
              - "segments" (list[str]): A list of individual markdown-formatted
                                        links for each grounding chunk.
              - "segment_string" (str): A concatenated string of all markdown-
                                        formatted links for the citation.
              Returns an empty list if no valid candidates or grounding supports
              are found, or if essential data is missing.
    """
    citations = []

    # Ensure response and necessary nested structures are present
    if not response or not response.candidates:
        return citations

    candidate = response.candidates[0]
    if (
        not hasattr(candidate, "grounding_metadata")
        or not candidate.grounding_metadata
        or not hasattr(candidate.grounding_metadata, "grounding_supports")
    ):
        return citations

    for support in candidate.grounding_metadata.grounding_supports:
        citation = {}

        # Ensure segment information is present
        if not hasattr(support, "segment") or support.segment is None:
            continue  # Skip this support if segment info is missing

        start_index = (
            support.segment.start_index
            if support.segment.start_index is not None
            else 0
        )

        # Ensure end_index is present to form a valid segment
        if support.segment.end_index is None:
            continue  # Skip if end_index is missing, as it's crucial

        # Add 1 to end_index to make it an exclusive end for slicing/range purposes
        # (assuming the API provides an inclusive end_index)
        citation["start_index"] = start_index
        citation["end_index"] = support.segment.end_index

        citation["segments"] = []
        if (
            hasattr(support, "grounding_chunk_indices")
            and support.grounding_chunk_indices
        ):
            for ind in support.grounding_chunk_indices:
                try:
                    chunk = candidate.grounding_metadata.grounding_chunks[ind]
                    resolved_url = resolved_urls_map.get(chunk.web.uri, None)
                    citation["segments"].append(
                        {
                            "label": chunk.web.title.split(".")[:-1][0],
                            "short_url": resolved_url,
                            "value": chunk.web.uri,
                        }
                    )
                except (IndexError, AttributeError, NameError):
                    # Handle cases where chunk, web, uri, or resolved_map might be problematic
                    # For simplicity, we'll just skip adding this particular segment link
                    # In a production system, you might want to log this.
                    pass
        citations.append(citation)
    return citations


## Prompts

In [9]:
# ==========================================
# 1. RESEARCH AGENT PROMPTS
# ==========================================
query_writer_instructions = """Your goal is to generate sophisticated and diverse web search queries. These queries are intended for an advanced automated web research tool capable of analyzing complex results, following links, and synthesizing information.

Instructions:
- Always prefer a single search query, only add another query if the original question requests multiple aspects or elements and one query is not enough.
- Each query should focus on one specific aspect of the original question.
- Don't produce more than {number_queries} queries.
- Queries should be diverse, if the topic is broad, generate more than 1 query.
- Don't generate multiple similar queries, 1 is enough.
- Query should ensure that the most current information is gathered. The current date is {current_date}.

Format: 
- Format your response as a JSON object with ALL three of these exact keys:
   - "rationale": Brief explanation of why these queries are relevant
   - "query": A list of search queries

Example:

Topic: What revenue grew more last year apple stock or the number of people buying an iphone
```json
{{
    "rationale": "To answer this comparative growth question accurately, we need specific data points on Apple's stock performance and iPhone sales metrics. These queries target the precise financial information needed: company revenue trends, product-specific unit sales figures, and stock price movement over the same fiscal period for direct comparison.",
    "query": ["Apple total revenue growth fiscal year 2024", "iPhone unit sales growth fiscal year 2024", "Apple stock price growth fiscal year 2024"],
}}
```

Context: {research_topic}"""

web_searcher_instructions = """Conduct targeted Google Searches to gather the most recent, credible information on "{research_topic}" and synthesize it into a verifiable text artifact.

Instructions:
- Query should ensure that the most current information is gathered. The current date is {current_date}.
- Conduct multiple, diverse searches to gather comprehensive information.
- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.
- The output should be a well-written summary or report based on your search findings. 
- Only include the information found in the search results, don't make up any information.

Research Topic:
{research_topic}
"""

reflection_instructions = """You are an expert research assistant analyzing summaries about "{research_topic}".

Instructions:
- Identify knowledge gaps or areas that need deeper exploration and generate a follow-up query. (1 or multiple).
- If provided summaries are sufficient to answer the user's question, don't generate a follow-up query.
- If there is a knowledge gap, generate a follow-up query that would help expand your understanding.
- Focus on technical details, implementation specifics, or emerging trends that weren't fully covered.

Requirements:
- Ensure the follow-up query is self-contained and includes necessary context for web search.

Output Format:
- Format your response as a JSON object with these exact keys:
   - "is_sufficient": true or false
   - "knowledge_gap": Describe what information is missing or needs clarification
   - "follow_up_queries": Write a specific question to address this gap

Example:
```json
{{
    "is_sufficient": true, // or false
    "knowledge_gap": "The summary lacks information about performance metrics and benchmarks", // "" if is_sufficient is true
    "follow_up_queries": ["What are typical performance benchmarks and metrics used to evaluate [specific technology]?"] // [] if is_sufficient is true
}}
```

Reflect carefully on the Summaries to identify knowledge gaps and produce a follow-up query. Then, produce your output following this JSON format:

Summaries:
{summaries}
"""

answer_instructions = """Generate a high-quality answer to the user's question based on the provided summaries.

Instructions:
- The current date is {current_date}.
- You are the final step of a multi-step research process, don't mention that you are the final step. 
- You have access to all the information gathered from the previous steps.
- You have access to the user's question.
- Generate a high-quality answer to the user's question based on the provided summaries and the user's question.
- You MUST include all the citations from the summaries in the answer correctly.
- You MUST format your answer in markdown and include links formatted as [label](link)!
- Do not provide number labels. Use the proper link label.

User Question:
"{users_question}"

User Context:
- {research_topic}

Summaries:
{summaries}"""



## Schemas

In [10]:
from pydantic import BaseModel, Field
from typing import List

class SearchQueryList(BaseModel):
    query: List[str] = Field(
        description="A list of search queries to be used for web research."
    )
    rationale: str = Field(
        description="A brief explanation of why these queries are relevant to the research topic."
    )

class Reflection(BaseModel):
    is_sufficient: bool = Field(
        description="Whether the provided summaries are sufficient to answer the user's question."
    )
    knowledge_gap: str = Field(
        description="A description of what information is missing or needs clarification."
    )
    follow_up_queries: List[str] = Field(
        description="A list of follow-up queries to address the knowledge gap."
    )

## Nodes

In [11]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from google.genai import Client
from langgraph.types import Send

from langchain_core.messages import AIMessage
from logging_decorator import log_node_execution, log_node

from dotenv import load_dotenv

load_dotenv()

genai_client = Client(api_key=os.getenv("GEMINI_API_KEY"))

@log_node_execution("🔍 Query Generation")
def generate_queries(state: OverallState, config: RunnableConfig) -> QueryGenerationState:
    """LangGraph node that generates search queries based on the User's question.

    Uses Gemini 2.0 Flash to create an optimized search query for web research based on
    the User's question.

    Args:
        state: Current graph state containing the User's question
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated query
    """
    # Get configurable values
    configurable = Configuration.from_runnable_config(config)
    model = configurable.query_generator_model
    
    # check for custom initial search query count
    if state.get("initial_search_query_count") is None:
        state["initial_search_query_count"] = configurable.number_of_initial_queries or 3

    if 'gemini' in model.lower():
        llm = ChatGoogleGenerativeAI(
            model=model,
            temperature=0,
            max_retries=2,
            api_key=os.getenv("GEMINI_API_KEY"),
        )
    elif ('gpt' in model.lower()) | ('o3' in model.lower()) | ('o4' in model.lower()) | ('o1' in model.lower()):
        llm = ChatOpenAI(
            model=model,
            temperature=1,
            max_retries=2,
            api_key=os.getenv("OPENAI_API_KEY")
        )

    structured_llm = llm.with_structured_output(SearchQueryList)

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = query_writer_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        number_queries=state["initial_search_query_count"],
    )
    # Generate the search queries
    result = structured_llm.invoke(formatted_prompt)
    return {"query_list": result.query}

@log_node_execution("🌐 Web Research")
def web_research(state: WebSearchState, config: RunnableConfig) -> OverallState:
    """LangGraph node that performs web research using the native Google Search API tool.

    Executes a web search using the native Google Search API tool in combination with Gemini 2.0 Flash.

    Args:
        state: Current graph state containing the search query and research loop count
        config: Configuration for the runnable, including search API settings

    Returns:
        Dictionary with state update, including sources_gathered, research_loop_count, and web_research_results
    """
    # Get configurable values
    configurable = Configuration.from_runnable_config(config)
    
    # Configure
    formatted_prompt = web_searcher_instructions.format(
        current_date=get_current_date(),
        research_topic=state["search_query"],
    )
    
    # Get model name and ensure it works with Google Search
    model = configurable.query_generator_model or 'gemini-2.0-flash'

    if 'gemini' not in model:
        print("Model not supported for search. Defaulting to google model.")
        model = 'gemini-2.0-flash'
    
    # # For Google GenAI native client, use a model that supports search
    # if model == 'gemini-2.0-flash':
    #     model = 'models/gemini-2.0-flash-exp'  # Use experimental version that supports search
    # elif not model.startswith('models/'):
    #     model = f'models/{model}'

    # Uses the google genai client as the langchain client doesn't return grounding metadata
    response = genai_client.models.generate_content(
        model=model,
        contents=formatted_prompt,
        config={
            "tools": [{"google_search": {}}],
            "temperature": 0,
        },
    )
    # resolve the urls to short urls for saving tokens and time
    resolved_urls = resolve_urls(
        response.candidates[0].grounding_metadata.grounding_chunks, state["id"]
    )

    # Gets the citations and adds them to the generated text
    citations = get_citations(response,resolved_urls)
    modified_text = insert_citation_markers(response.text, citations)
    sources_gathered = [item for citation in citations for item in citation["segments"]]

    return {
        "sources_gathered": sources_gathered,
        "search_query": [state["search_query"]],
        "web_research_result": [modified_text],
    }

@log_node_execution("🤔 Reflection")
def reflection(state: OverallState, config: RunnableConfig) -> ReflectionState:
    """LangGraph node that identifies knowledge gaps and generates potential follow-up queries.

    Analyzes the current summary to identify areas for further research and generates
    potential follow-up queries. Uses structured output to extract
    the follow-up query in JSON format.

    Args:
        state: Current graph state containing the running summary and research topic
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated follow-up query
    """
    configurable = Configuration.from_runnable_config(config)
    # Increment the research loop count and get the reasoning model
    state["research_loop_count"] = state.get("research_loop_count", 0) + 1
    reasoning_model = configurable.reasoning_model or {}

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = reflection_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        summaries="\n\n---\n\n".join(state["web_research_result"]),
    )
    if 'gemini' in reasoning_model.lower():
        llm = ChatGoogleGenerativeAI(
            model=reasoning_model,
            temperature=0,
            max_retries=2,
            api_key=os.getenv("GEMINI_API_KEY"),
        )
    elif ('gpt' in reasoning_model.lower()) | ('o3' in reasoning_model.lower()) | ('o4' in reasoning_model.lower()) | ('o1' in reasoning_model.lower()):
        llm = ChatOpenAI(
            model=reasoning_model,
            temperature=1,
            max_retries=2,
            api_key=os.getenv("OPENAI_API_KEY")
        )
        
    result = llm.with_structured_output(Reflection).invoke(formatted_prompt)

    return {
        "is_sufficient": result.is_sufficient,
        "knowledge_gap": result.knowledge_gap,
        "follow_up_queries": result.follow_up_queries,
        "research_loop_count": state["research_loop_count"],
        "number_of_ran_queries": len(state["search_query"]),
    }

@log_node_execution("📋 Finalize Answer")
def finalize_answer(state: OverallState, config: RunnableConfig) -> OverallState:
    """LangGraph node that finalizes the research summary.

    Prepares the final output by combining research results into a well-structured
    research report with proper citations and working markdown links.

    Args:
        state: Current graph state containing the running summary and sources gathered

    Returns:
        Dictionary with state update, including the final answer with working citations
    """
    configurable = Configuration.from_runnable_config(config)
    reasoning_model = configurable.reasoning_model or {}

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = answer_instructions.format(
        current_date=current_date,
        users_question=state['messages'][0],
        research_topic=get_research_topic(state["messages"]),
        summaries="\n---\n\n".join(state["web_research_result"]),
    )

    if 'gemini' in reasoning_model.lower():
        llm = ChatGoogleGenerativeAI(
            model=reasoning_model,
            temperature=0,
            max_retries=2,
            api_key=os.getenv("GEMINI_API_KEY"),
        )
    elif ('gpt' in reasoning_model.lower()) | ('o3' in reasoning_model.lower()) | ('o4' in reasoning_model.lower()) | ('o1' in reasoning_model.lower()):
        llm = ChatOpenAI(
            model=reasoning_model,
            temperature=1,
            max_retries=2,
            api_key=os.getenv("OPENAI_API_KEY")
        )
    else:
        "Model not supported. Only use Google or OpenAI models."

    result = llm.invoke(formatted_prompt)

    # Collect unique sources that are actually used in the final content
    unique_sources = []
    for source in state["sources_gathered"]:
        if source["short_url"] in result.content:
            result.content = result.content.replace(
                f'({source["short_url"]})', f'({source["value"]})'
            )
            unique_sources.append(source)

    return {
        "messages": [AIMessage(content=result.content)],
        "sources_used": unique_sources,
    }

# ==========================================
# CONDITIONAL EDGES
# ==========================================

@log_node
def continue_to_web_research(state: QueryGenerationState):
    """LangGraph node that sends the search queries to the web research node.

    This is used to spawn n number of web research nodes, one for each search query.
    """
    return [
        Send("web_research", {"search_query": search_query, "id": int(idx)})
        for idx, search_query in enumerate(state["query_list"])
    ]

@log_node
def evaluate_research(
    state: ReflectionState,
    config: RunnableConfig,
) -> OverallState:
    """LangGraph routing function that determines the next step in the research flow.

    Controls the research loop by deciding whether to continue gathering information
    or to finalize the summary based on the configured maximum number of research loops.

    Args:
        state: Current graph state containing the research loop count
        config: Configuration for the runnable, including max_research_loops setting

    Returns:
        String literal indicating the next node to visit ("web_research" or "finalize_summary")
    """
    configurable = Configuration.from_runnable_config(config)
    max_research_loops = (
        state.get("max_research_loops")
        if state.get("max_research_loops") is not None
        else configurable.max_research_loops
    )
    if state["is_sufficient"] or state["research_loop_count"] >= max_research_loops:
        return "finalize_answer"
    else:
        return [
            Send(
                "web_research",
                {
                    "search_query": follow_up_query,
                    "id": state["number_of_ran_queries"] + int(idx),
                },
            )
            for idx, follow_up_query in enumerate(state["follow_up_queries"])
        ]

## Graph

In [12]:
from langgraph.graph import StateGraph, START, END
from state import OverallState

graph_builder = StateGraph(OverallState, config_schema=Configuration)

# ==========================================
# Define Nodes
# ==========================================

graph_builder.add_node("generate_queries",generate_queries)
graph_builder.add_node("web_research",web_research)
graph_builder.add_node("reflection",reflection)
graph_builder.add_node("finalize_answer",finalize_answer)


# ==========================================
# Define Edges
# ==========================================

graph_builder.add_edge(START, "generate_queries")  
graph_builder.add_conditional_edges(
    "generate_queries", continue_to_web_research, ["web_research"]
)
graph_builder.add_edge('web_research','reflection')
graph_builder.add_conditional_edges(
    "reflection", evaluate_research, ["web_research", "finalize_answer"]
)
graph_builder.add_edge("web_research", END)

# ==========================================
# Compile The Graph
# ==========================================

graph = graph_builder.compile()

# Test Sustainability Agent

In [13]:
prompt = """Research {company_name}'s adoption of innovative sustainable technologies and provide an overall score (0-100) based on the criteria below:

## RESEARCH AREAS:

1. **Key Technologies Adopted (25 points)**: What specific sustainable technologies (renewable energy, energy efficiency, carbon management, sustainable materials) has {company_name} implemented?

2. **Innovation Leadership (25 points)**: What cutting-edge sustainable technology initiatives, R&D projects, or partnerships is {company_name} pursuing?

3. **Measurable Impact (25 points)**: What quantifiable environmental and business impacts have resulted from their sustainable technology investments?

4. **Future Strategy (15 points)**: What are {company_name}'s documented commitments and roadmap for future sustainable technology adoption?

5. **Industry Position (10 points)**: How does {company_name} compare to competitors in sustainable technology leadership and adoption?

## SCORING CRITERIA (Total: 100 points):

**Technology Adoption (25 points):**
- 20-25: Multiple breakthrough technologies implemented at scale across operations
- 15-19: Several innovative technologies deployed with significant operational integration
- 10-14: Some sustainable technologies adopted but limited scope or innovation
- 5-9: Basic sustainable technology implementations, mostly conventional solutions
- 0-4: Minimal or no significant sustainable technology adoption

**Innovation Leadership (25 points):**
- 20-25: Leading industry with proprietary sustainable tech development, major R&D investments, breakthrough patents
- 15-19: Strong innovation program with partnerships, pilot projects, and emerging tech investments
- 10-14: Moderate innovation efforts, some R&D or partnerships in sustainable tech
- 5-9: Limited innovation activities, following industry trends rather than leading
- 0-4: No significant sustainable technology innovation or R&D

**Measurable Impact (25 points):**
- 20-25: Documented major environmental improvements (>50% emissions reduction, significant resource savings)
- 15-19: Strong measurable impact (25-50% improvements in key metrics)
- 10-14: Moderate documented impact (10-25% improvements)
- 5-9: Some measurable benefits but limited scope or impact
- 0-4: No clear quantifiable environmental or efficiency benefits

**Future Strategy (15 points):**
- 13-15: Comprehensive roadmap with specific targets, timelines, and major investments committed
- 10-12: Clear strategy with some specific commitments and targets
- 7-9: General sustainability commitments with limited specific tech focus
- 4-6: Vague future plans, minimal concrete commitments
- 0-3: No clear future sustainable technology strategy

**Industry Position (10 points):**
- 9-10: Industry leader, setting standards, widely recognized for sustainable tech leadership
- 7-8: Among top performers in industry sector
- 5-6: Average compared to industry peers
- 3-4: Below average, following rather than leading
- 0-2: Laggard in industry for sustainable technology adoption

## OUTPUT REQUIREMENTS:
- Provide detailed findings for each research area
- Give specific scores for each category with justification
- Calculate and present final overall score (0-100)
- Include recent examples (2022-2025), specific metrics, and credible sources
- Focus on innovation and leadership rather than basic compliance 

## FINAL ASSESSMENT FORMAT:
**SUSTAINABLE TECHNOLOGY ADOPTION SCORE: [X]/100**

Technology Adoption: [X]/25
Innovation Leadership: [X]/25  
Measurable Impact: [X]/25
Future Strategy: [X]/15
Industry Position: [X]/10

**Overall Assessment**: [Brief summary of company's sustainable tech position]"""

In [14]:
company = '3M'
formated_prompt = prompt.format(company_name=company)

In [15]:
formated_prompt

"Research 3M's adoption of innovative sustainable technologies and provide an overall score (0-100) based on the criteria below:\n\n## RESEARCH AREAS:\n\n1. **Key Technologies Adopted (25 points)**: What specific sustainable technologies (renewable energy, energy efficiency, carbon management, sustainable materials) has 3M implemented?\n\n2. **Innovation Leadership (25 points)**: What cutting-edge sustainable technology initiatives, R&D projects, or partnerships is 3M pursuing?\n\n3. **Measurable Impact (25 points)**: What quantifiable environmental and business impacts have resulted from their sustainable technology investments?\n\n4. **Future Strategy (15 points)**: What are 3M's documented commitments and roadmap for future sustainable technology adoption?\n\n5. **Industry Position (10 points)**: How does 3M compare to competitors in sustainable technology leadership and adoption?\n\n## SCORING CRITERIA (Total: 100 points):\n\n**Technology Adoption (25 points):**\n- 20-25: Multiple 

In [16]:
config = Configuration

state = graph.invoke(
        {"messages": [{"role": "user", "content": formated_prompt}]}
    )
report = state["messages"][-1].content

2025-08-21 07:37:40,955 - research_agent - INFO - 🚀 Starting 🔍 Query Generation
2025-08-21 07:37:40,959 - research_agent - INFO -    State Counts: {'search_queries_count': 0, 'research_results_count': 0, 'sources_count': 0, 'research_loop': 0, 'messages_count': 1}
2025-08-21 07:37:40,960 - research_agent - INFO -    Input Query: Research 3M's adoption of innovative sustainable technologies and provide an overall score (0-100) based on the criteria below:

## RESEARCH AREAS:

1. **Key Technologies Adopted (25 points)**: What specific sustainable technologies (renewable energy, energy efficiency, carbon management, sustainabl...
2025-08-21 07:37:41,011 - research_agent - INFO - 🤖 🔍 Query Generation: Starting LLM call...
2025-08-21 07:37:42,812 - research_agent - INFO - ✅ Completed 🔍 Query Generation in 1.80s
2025-08-21 07:37:42,813 - research_agent - INFO -    Generated Queries: ['3M sustainable technology initiatives 2022-2025', '3M environmental impact metrics 2022-2025', '3M sustainab

In [6]:
state['messages'][0]

HumanMessage(content="Research 3M's adoption of innovative sustainable technologies and provide an overall score (0-100) based on the criteria below:\n\n## RESEARCH AREAS:\n\n1. **Key Technologies Adopted (25 points)**: What specific sustainable technologies (renewable energy, energy efficiency, carbon management, sustainable materials) has 3M implemented?\n\n2. **Innovation Leadership (25 points)**: What cutting-edge sustainable technology initiatives, R&D projects, or partnerships is 3M pursuing?\n\n3. **Measurable Impact (25 points)**: What quantifiable environmental and business impacts have resulted from their sustainable technology investments?\n\n4. **Future Strategy (15 points)**: What are 3M's documented commitments and roadmap for future sustainable technology adoption?\n\n5. **Industry Position (10 points)**: How does 3M compare to competitors in sustainable technology leadership and adoption?\n\n## SCORING CRITERIA (Total: 100 points):\n\n**Technology Adoption (25 points):*

In [20]:
from IPython.display import Markdown, display

# This will render markdown properly
display(Markdown(report))

Below is a focused, evidence-based assessment of 3M’s adoption of innovative sustainable technologies (2022–2025), scored against your stated criteria and supported with the citations provided in the research summaries.

SUSTAINABLE TECHNOLOGY ADOPTION SCORE: 83/100

Technology Adoption: 22/25  
Innovation Leadership: 18/25  
Measurable Impact: 22/25  
Future Strategy: 13/15  
Industry Position: 8/10

Overall Assessment: 3M is a strong performer on sustainable technology adoption. It has deployed multiple operational-scale solutions (renewable electricity, energy-efficiency measures, water purification, plastic-reduction/circularity, wind/solar film products and advanced filtration media) and is investing in next‑generation climate technologies (carbon sorbent materials with Svante, catalysts for green hydrogen). Its emissions and resource‑use metrics show material, documented improvements; near-term targets are SBTi-validated and tied to public investment commitments. 3M ranks among the top peers for breadth and scale of deployment, though some R&D investment trends and remaining scope‑3 challenges limit a top-tier innovation leadership score.

Detailed findings, evidence and scoring justification

Key technologies adopted (25 points) — Score: 22/25
Findings and examples
- Renewable electricity and PPAs: 3M reported achieving ~56.2% renewable electricity across global operations in 2023 and has multiple PPAs (e.g., Gunsight wind PPA and other regional contracts) supporting operational renewable energy uptake and its RE100 membership commitment to 100% renewables by 2050 [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_tZ7qV6oOyYdAoU1i5qgKjYbd8GOt6GG6guRhyeoDR9p1Vpilwf1m1tAG0uygPGk2cjeFnfPYu6mI7Y2-nnU514ToRlbiSL1TQylK1c_V9ah8GwurzvYbCKOBV-2B5qghpq_JuHbXye5YcQ==) [prnewswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG0poW4F6ik2hzAdozUl1b0mb1U7IK1MjYw8wlk0LJzMMFeu7O0uiZ_tX3euBMKp1DZI9V4L4tDqNKTbKWO8Jbw1HHc7SdxlmtBIDtw9-jiXIKF294Z1vN7F2JlY6VFrbgdLBPWTE5twkkKSSinkii3N8t7MGzvskhC5jrvOQNAi0RFTwbmtJ5V9AizD36MPaFSnZiTfXy9NpUMtqVPC5H7) [renewablesnow](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEDELSywgCISEY5TeT340FTHUsOYMxnMC85MUPiFevNRfqG1EEfCK4fbU7HMjPWML-fC5tyKNi07xB-Bm2T4NKwgds5VV_fpFo7bny8jZLINwaL7pDBSjCXziU-g1n5zzAJ2c8CpF-GbXnFPxXff2pbHOK7UYnOakFUQmT5xt8z3VPvIMbXU-e6) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLJpCD_1KfwdICe191Ztl_l4WkG1KfrtsL2oXcWKUuol90GYPktytFZvpbsb5tyLnz_LfoNd23DZAzXCrU9Mi3kKul4JswxJtvQvxwn_igM221ViS0MgqmbSTj0UxE88SMgzi_AXz7PCGgyhVH9XpDfxyUJ2JN-BllUyibEQGwVugRA6kDCIgjsddFFSMLK_3s0RGODCnXUenC) [sustainable-energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlsOUgaFgb88gD6WC8wF38zGPfdb6ahnRG_Mk8dEYZFC_I3faR1x_4_PCXJa3xdK6Di_Qc0Sn3AuWHA5Qpl2SvURaObIx_IwvKd2OHHEbYxkFrCf6-yJH6KdORA7X1IkmcIAkjoU2Eqll0Ilum8dcfZtV_Lo5RMfq4-pBQLyFe6sEOgJM3Wvo0jUnfEdyt6nQFOvGy5HMwEz5fSNk0zr1czUJV3k1G).
- Energy-efficiency & wind: 3M markets turbine blade vortex generators shown to raise AEP (1.6–3% reported increases) and supplies energy-management and efficiency solutions across manufacturing sites [edf-re](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGjNSIhzmvKm8-mdlFFX0jWxujYxrglurO8nRDeQt5uDAVCSjlhZJ7Qth5ea4SJfyDwhy_5MM5SHI7qw_xba9Zt4AweS_F_HEAlTxhzswQDgByJgOFth0dEYBa4RHvz0rjCCxbU09tTF5p-zGWblb3QEnqRjGgW2i18YfYyqZ7t4I_8Sf9Kw1ABaUyMv8k3bk0=) [nawindpower](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEKw9UMBWyRAAiyr9KXXLrrzVODAgxSAQme3q0Kqsy-RG8lIF0zb5DWEkWDbtZIa0nqonOilmZjTtVS9iygeKCskl7_VYR5hr-MnDOLgzdkUGkLd4espPTVa8GabtxWG-KzIw-dE0Cm8Ao66R9J2gry_x3CCUX8ncLLw2S7ivdvqnCU8fmfCg==) [windtech-international](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGUvjHwuxGCvQ7V1RwtD--xg0xsqYPjpJq9_u5OBxSSo_KyQKuidIiqnk4cb1MVsz-g0uiWRw7mZGlvowz_fTy3vZMt8TURV-70e1U29io6XN0abb8qmLLLD-RpSQ6QZEYsDRf-nnzN3qZnwQkDjbf2dd624Ad9Zo8f0Z7Blv56AUjxdwT5gCnOljAuTw9uu_k0He0nJ0sqnA4Wvnqr0yPDwXerdm6f32lewWTC).
- Carbon management / CCUS materials: 3M is collaborating with Svante to produce sorbent materials (“Sorbent-on-a-Roll”) for CO2 capture (regeneration by steam heat, designed for scale manufacture) and is developing nanostructured iridium catalyst powders for electrolysis (green hydrogen) [svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG2ulzRbdGnhF2-19zrppybobjZacJHfHvvY8KN4D9uOmL5jJG1krvPiOr_-DktKu4_AabvSBjBZ9u9i7YlwxnkvJySUhaUWzmHpfiLqYZ89BQpVd3o96apNYtDvF12zGrXbIvvY4jiUQnIY7Xm2Pk018jpeqC6EutgQUugaRIKyUK12dR_znGiAdIsdqG1uSeAtupTycLWqsGO) [3blmedia](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGTzFA3XCWGMZlGvaFBq6RlsQfRRLRnwyRYzA6XTTNoNmmC9eDVCYu38RENfa6o5o5RLg2aL0oatwesCA7p20telAO7a2SQvyL_6mo_xRYWcJNr7drfRxcIxmqhbaza3CxXrueua5Xf02UE-jPpjK9l8Tvze7i9QG58BnfQOgIcRKWOqCk=) [environmentenergyleader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyxCGkuRYZ9if4m2PAhpZCCwfzuFWxzC-_GUGjKBjPEEkWY7MICwVlNF6uBPRFOrcdMR9hvWiT9nHxjn6Sc94LK0w79QXQnAHU08tKykq-f6LLz4K53Yu4ROBEJnz2fEJTWIuEjVVyb-W6kb78AJtJ8uMP_MBUS9tOt8xkP4DhTJ-wxSrAFsOJtRjwQw16ve5B0Ga3r_nrlYaT-peZYYz_).
- Water purification and management technologies: 3M is installing advanced water purification at its largest water use sites (part of a targeted program to reduce water use and return higher‑quality water) and has published measurable water-efficiency gains [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFOLw3KkVQL00jLIvpWvQqfXGGSTFQxka7HaG-nKWuZj64ZozMPARj5p1lgn-grH_IwoViyBkYK--OEkLmjhKoQ8-Sx7fniSQLqitdBEzqYNN9vumJktl53gHflYa_FPq1PdpJ4by09mcU3UGjT78RDS-MOlywIrtXKAv_K_J4qCC7sM1cUnusnetsU6PZa3TgyxGDpjd2DdFs3laPxfFp5psbA0yxwKlo3Jh9_3p8=) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLJpCD_1KfwdICe191Ztl_l4WkG1KfrtsL2oXcWKUuol90GYPktytFZvpbsb5tyLnz_LfoNd23DZAzXCrU9Mi3kKul4JswxJtvQvxwn_igM221ViS0MgqmbSTj0UxE88SMgzi_AXz7PCGgyhVH9XpDfxyUJ2JN-BllUyibEQGwVugRA6kDCIgjsddFFSMLK_3s0RGODCnXUenC).
- Sustainable materials & circularity: large reductions in virgin fossil‑based plastics use, adoption of recycled content and bio-based options, and product-level Sustainability Value Commitments (SVCs) for every new product [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHiNIo8KbfkmvPZX5P5RiuSpeK6EZq4Fd_wmWdhqe-rhSlQfuxJZSK1Ut6XGTA9HpUC7vfYE4Zb0Gnq-T4hywyKfFalRi3AlBkJxnlO8OhdxxOENIKVejdmUE8-K0fO5XTC47aK1W970JHDQrqiz71ighxGRa8uzEe5TkMx7Q5Wsw==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEQ31OPsEuj1Fh4kuhwW3kCMmuBvvW7vcyBDLVxcHdrfFQXGf5-h81_UWnd1vC7x9tWKT6vDqRFs89EKehyveJFH_L7vQgWzFYuLTIoRL4cwJ4SaDKyjkHgWAwLGLjusuSp5gXRmQ==) [energyindustryreview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG4JdH5UCbyM-oFJ5W3aRhI-IINM6S1vaE2zR2on6llAPVx4S5FgHJuHVGFYPGvSb7Tl4Vlh0aD9sw55hOI3Ve0RJOQlQMAEdsr_l_Stk-oUCgxQUvSWNCWmirdGkyj9Lq5SAIfdGLGmgw4iIzfvupJHpdobMpsAXKmzAPiVvHw8YSAyxDVET3dfi0DFOHbGbZROkTdydZIoQ34-d_0L1fyXSyYuKRGd3mFwT305HxRr7kv) [packagingguruji](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEF5dJ45hUZzoFUfTkvlNTV4zj6iG_9vWTXEdsqIgpCWNp7eSKqBx0URSg1xXe2MREKXZFp2da3JDM6VhCl2nY4Jab7sbJGZ-FrPOl3S-du1rs9aLb_ghh2Ft-9yV-44atgkVhAroBbrrhyg4IEOJM1NHdSJbvObA==).
- PFAS phase-out: 3M has committed to exit PFAS manufacturing and discontinue PFAS use across its portfolio by end of 2025 — a major materials/portfolio transformation [remediation-technology](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE8EEWH_IzGJbou01debNxMryDoMx-RJdmjSPutFxMsC1Gwlo0wVijbSxy-sAty04mJ3_tpT5xQLKQKvOO5YTv7_lYDiMyJ04qtpvNj5VXsDm3aydyp_iV2hG2pT6ETfl1fH4Xndr1POIjZF1TvPALmcubGYuIUDym1LOSpPb2-Q7MihW6Ikk0ZrmzzSKs9hI97wahRXE5ei82eEA==) [trellis](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFEooTQqeJeaZpivQPAl3KCIThPpbdnAeNlYKH7gX6bxD2UUTRJglNcl-6GoiRUHR74jLXOqglJXJps3RzLAaYr4fiQYu2H7n--GvKRewtebVlLPqtIVQ3bk0tJfjZeBfbd3PqEuQYYbB0biTF2hTmyUFROa4aZx0OyyyWmuHws0hBmZsET) [investigatemidwest](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQErEpvygtm0iRuGWN1akPvi-569pFfXoQTFf2jeMT8b7m8vFegam-A7XWrmv4hkGPHwvOCquleeMdFevYL2zOb0wplv2IJlz3fscWHaUHc_xuyPOoNO_YSAtrgpd9AFruaipwwIRNJRi8IZuijinu3hE8EhoiCaU57WgL7dHdDc3ae5Y16LJkb8x3VDVdRqDLXHGh6_5QbCfWUOOyzkiQVy) [manufacturingdive](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEkEeheEFOGptqmzNjYL-A6gU9yEnd_snO2gUayY0Jus5W5JWVyqFVpHhXzoJ3unS0H3yEGEJeM0zT8U0ban4r4yRe3_C1nvabnwp-MTDmp1TsozliyTCkw-Z3cn-Qy4DrJUc8Nygv_Zv6xsdoH9TPPLP0GhbwtabHrQirDke_YK88Z9unMOhUd8gOX0okXHJKfTgK-lbzXcqYe0ofgc1W2692jVKlbhQM1SLAt).

Why 22/25
3M has implemented multiple innovative sustainable technologies at operational scale (renewable electricity >50% across operations, advanced water systems, circular‑materials progress, CCUS material collaborations, wind/solar product lines). The company earns near‑top marks for breadth and scale; a small deduction reflects that some technologies (e.g., commercial carbon capture deployments) are still in partnership/pilot stages rather than full, company‑led commercial rollouts everywhere.

Key citations supporting this section: [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_tZ7qV6oOyYdAoU1i5qgKjYbd8GOt6GG6guRhyeoDR9p1Vpilwf1m1tAG0uygPGk2cjeFnfPYu6mI7Y2-nnU514ToRlbiSL1TQylK1c_V9ah8GwurzvYbCKOBV-2B5qghpq_JuHbXye5YcQ==) [prnewswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG0poW4F6ik2hzAdozUl1b0mb1U7IK1MjYw8wlk0LJzMMFeu7O0uiZ_tX3euBMKp1DZI9V4L4tDqNKTbKWO8Jbw1HHc7SdxlmtBIDtw9-jiXIKF294Z1vN7F2JlY6VFrbgdLBPWTE5twkkKSSinkii3N8t7MGzvskhC5jrvOQNAi0RFTwbmtJ5V9AizD36MPaFSnZiTfXy9NpUMtqVPC5H7) [renewablesnow](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEDELSywgCISEY5TeT340FTHUsOYMxnMC85MUPiFevNRfqG1EEfCK4fbU7HMjPWML-fC5tyKNi07xB-Bm2T4NKwgds5VV_fpFo7bny8jZLINwaL7pDBSjCXziU-g1n5zzAJ2c8CpF-GbXnFPxXff2pbHOK7UYnOakFUQmT5xt8z3VPvIMbXU-e6) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLJpCD_1KfwdICe191Ztl_l4WkG1KfrtsL2oXcWKUuol90GYPktytFZvpbsb5tyLnz_LfoNd23DZAzXCrU9Mi3kKul4JswxJtvQvxwn_igM221ViS0MgqmbSTj0UxE88SMgzi_AXz7PCGgyhVH9XpDfxyUJ2JN-BllUyibEQGwVugRA6kDCIgjsddFFSMLK_3s0RGODCnXUenC) [sustainable-energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlsOUgaFgb88gD6WC8wF38zGPfdb6ahnRG_Mk8dEYZFC_I3faR1x_4_PCXJa3xdK6Di_Qc0Sn3AuWHA5Qpl2SvURaObIx_IwvKd2OHHEbYxkFrCf6-yJH6KdORA7X1IkmcIAkjoU2Eqll0Ilum8dcfZtV_Lo5RMfq4-pBQLyFe6sEOgJM3Wvo0jUnfEdyt6nQFOvGy5HMwEz5fSNk0zr1czUJV3k1G) [svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG2ulzRbdGnhF2-19zrppybobjZacJHfHvvY8KN4D9uOmL5jJG1krvPiOr_-DktKu4_AabvSBjBZ9u9i7YlwxnkvJySUhaUWzmHpfiLqYZ89BQpVd3o96apNYtDvF12zGrXbIvvY4jiUQnIY7Xm2Pk018jpeqC6EutgQUugaRIKyUK12dR_znGiAdIsdqG1uSeAtupTycLWqsGO) [3blmedia](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGTzFA3XCWGMZlGvaFBq6RlsQfRRLRnwyRYzA6XTTNoNmmC9eDVCYu38RENfa6o5o5RLg2aL0oatwesCA7p20telAO7a2SQvyL_6mo_xRYWcJNr7drfRxcIxmqhbaza3CxXrueua5Xf02UE-jPpjK9l8Tvze7i9QG58BnfQOgIcRKWOqCk=) [environmentenergyleader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyxCGkuRYZ9if4m2PAhpZCCwfzuFWxzC-_GUGjKBjPEEkWY7MICwVlNF6uBPRFOrcdMR9hvWiT9nHxjn6Sc94LK0w79QXQnAHU08tKykq-f6LLz4K53Yu4ROBEJnz2fEJTWIuEjVVyb-W6kb78AJtJ8uMP_MBUS9tOt8xkP4DhTJ-wxSrAFsOJtRjwQw16ve5B0Ga3r_nrlYaT-peZYYz_) [edf-re](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGjNSIhzmvKm8-mdlFFX0jWxujYxrglurO8nRDeQt5uDAVCSjlhZJ7Qth5ea4SJfyDwhy_5MM5SHI7qw_xba9Zt4AweS_F_HEAlTxhzswQDgByJgOFth0dEYBa4RHvz0rjCCxbU09tTF5p-zGWblb3QEnqRjGgW2i18YfYyqZ7t4I_8Sf9Kw1ABaUyMv8k3bk0=) [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFOLw3KkVQL00jLIvpWvQqfXGGSTFQxka7HaG-nKWuZj64ZozMPARj5p1lgn-grH_IwoViyBkYK--OEkLmjhKoQ8-Sx7fniSQLqitdBEzqYNN9vumJktl53gHflYa_FPq1PdpJ4by09mcU3UGjT78RDS-MOlywIrtXKAv_K_J4qCC7sM1cUnusnetsU6PZa3TgyxGDpjd2DdFs3laPxfFp5psbA0yxwKlo3Jh9_3p8=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHiNIo8KbfkmvPZX5P5RiuSpeK6EZq4Fd_wmWdhqe-rhSlQfuxJZSK1Ut6XGTA9HpUC7vfYE4Zb0Gnq-T4hywyKfFalRi3AlBkJxnlO8OhdxxOENIKVejdmUE8-K0fO5XTC47aK1W970JHDQrqiz71ighxGRa8uzEe5TkMx7Q5Wsw==) [energyindustryreview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG4JdH5UCbyM-oFJ5W3aRhI-IINM6S1vaE2zR2on6llAPVx4S5FgHJuHVGFYPGvSb7Tl4Vlh0aD9sw55hOI3Ve0RJOQlQMAEdsr_l_Stk-oUCgxQUvSWNCWmirdGkyj9Lq5SAIfdGLGmgw4iIzfvupJHpdobMpsAXKmzAPiVvHw8YSAyxDVET3dfi0DFOHbGbZROkTdydZIoQ34-d_0L1fyXSyYuKRGd3mFwT305HxRr7kv)

Innovation leadership (25 points) — Score: 18/25
Findings and examples
- R&D scale and strategic investment: 3M continues large R&D activity (multi‑billion planned R&D spending 2025–2027 to launch new products; R&D spending reported in the ~$1.1B range annually with recent increases and fluctuations) and a $1B multi‑decade sustainability investment fund to support climate/water goals [macrotrends](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpCrO-9fCHzn3_IP2r14XP1bD-IzFAXnj5N6O7Ff14F4qSznVp29a6SSd7ZIVxyHjQMGcASDfpYVABq1JoG6gY5FjELqy373PtgvLQROB3kNxL9q3wYs8oYXQPKI6tV1t065HVuHpWxZIRg-Ca9CmezjSH8fERZyMcZEoH6wJLkRInpEdd) [circularcollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzT9mdzm8yoYZc4ExpF9AQ6FtK4GdKbV_RB0xYEnaepct15N6fo1rd_oMqTmsfEPVBg5dbPxex25KiOCFt9sVrR8XxlFmqGVxzAj42e3tXzY78nHtDj6-nZEkXeJzA2qgaELAqIS2zHAAPTJAepkyv4ADPm_cji-CvjZbUmZxvQTogrsk8XwyLIz32_mxqfg==) [esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHNEbPLZp0dsc_ti4E5gw-7BaMsoi4DFKktRqRKrbuiaoBm2yH6UGZww-vwK98J7igdKdK67jupfTPjwTYmrfUjqdxXWamt9Ewt4CbMnImLo5cFTShpoNJwbbnCsipey0WdC2Pu6oAlvfDbrbFx-BiAbt1StsWKLxw7NRWYiO-IGRfCqSE_T2NssG9P1Di50Oe0hWEuoA==).
- Breakthrough and climate-focused innovations: collaborations (Svante) on CO2 capture sorbents and in‑house catalyst powders for electrolysis (green hydrogen) show proprietary material science capability and productization potential [svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFNex9LWz7hAoNsCp4baJAK5N8zgQ9ENPWBwi6qnbc_TeEjPD-Gbtn7Ovpci74dWcpy6cmYxGxCjDU5jFvP8LE4XToiPYFtwXqnX3F2Ft9G68VVIgExQrZ4rLb-TurTgIBnNDBdR1WaLqMBcXOT6XpV2c2ethF3UGapIRjhXHNHGgs7Hu8QseL5V6n3GV3gy7vyq8sXKc83bZkrUQ==) [3blmedia](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGTzFA3XCWGMZlGvaFBq6RlsQfRRLRnwyRYzA6XTTNoNmmC9eDVCYu38RENfa6o5o5RLg2aL0oatwesCA7p20telAO7a2SQvyL_6mo_xRYWcJNr7drfRxcIxmqhbaza3CxXrueua5Xf02UE-jPpjK9l8Tvze7i9QG58BnfQOgIcRKWOqCk=) [environmentenergyleader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyxCGkuRYZ9if4m2PAhpZCCwfzuFWxzC-_GUGjKBjPEEkWY7MICwVlNF6uBPRFOrcdMR9hvWiT9nHxjn6Sc94LK0w79QXQnAHU08tKykq-f6LLz4K53Yu4ROBEJnz2fEJTWIuEjVVyb-W6kb78AJtJ8uMP_MBUS9tOt8xkP4DhTJ-wxSrAFsOJtRjwQw16ve5B0Ga3r_nrlYaT-peZYYz_) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEosLDo-yYo6v9XB2uGTkQvmVimWuQlXbrqBJhKmm-jXKB-Rrz8chocIQ6wt8pkopZwPfHnUJmVg-QmvQizw1GdzoSW9GwExcflsKYW8nQ7FSLe5tuxPZAlhMZ1oD9KwCzn_ZR_YL4LYft5usTMsPN31Oktvl17jr6VdB62Rfpq7BNoeBJot7gdNdgINX-1fGV7iMTa_DzuFRrjg==).
- Partnerships and pilots: 3M Ventures’ participation in Svante financing, DOE Better Plants engagement, CEMEX alternative-fuel partnership, Water Resilience Coalition leadership membership, RE100 membership all demonstrate active collaboration to accelerate applied sustainable tech [svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFNex9LWz7hAoNsCp4baJAK5N8zgQ9ENPWBwi6qnbc_TeEjPD-Gbtn7Ovpci74dWcpy6cmYxGxCjDU5jFvP8LE4XToiPYFtwXqnX3F2Ft9G68VVIgExQrZ4rLb-TurTgIBnNDBdR1WaLqMBcXOT6XpV2c2ethF3UGapIRjhXHNHGgs7Hu8QseL5V6n3GV3gy7vyq8sXKc83bZkrUQ==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHR4NqeTMhABBcRbuaaHRehjCH2rkMVEW8No-mjQY_Yj9LYUaZG99Mr39-ubkm49dHlayZNnUo8gG9FmNZXV33d1dY3CxlEAzn7jf0HO-zAomwrI78ggXmDF3Mb_J58NWrJZZMbxE_WAj7gwAESIII0YTGn9CJHetf7_H3eMTs-PZjx0SJ-0rMUl7XN3NwXI5ku7m-3igGUi3YFGvaVSa2uilFVDxs=) [cemex](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHtCk0C_PKzI0tOiVdljzLCkkaqT8UxocWWuc12mfodDLGdmJt4S9SUwFI4GIWyDbVvwJrsy_OW-jQwCpHzP29KrYw4x9bpi2Wp8sKOqbg5xEOBHcLExr6qhDfI6tDQNW7EUWomqbDva-PFE2DPuh5VcUFzWEGso4A8NCoZQrBxBO8mI9Ql) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEosLDo-yYo6v9XB2uGTkQvmVimWuQlXbrqBJhKmm-jXKB-Rrz8chocIQ6wt8pkopZwPfHnUJmVg-QmvQizw1GdzoSW9GwExcflsKYW8nQ7FSLe5tuxPZAlhMZ1oD9KwCzn_ZR_YL4LYft5usTMsPN31Oktvl17jr6VdB62Rfpq7BNoeBJot7gdNdgINX-1fGV7iMTa_DzuFRrjg==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIKevCFAl69SkZaQX1hxt4x1FOv_CGVQS5qIEwzsXAFHIbq3vCgxDrDv-WknLO4Sl--1mkaEJW-4AMMsgsg_VNcwOM2KGzAqktx63GA1CkLs1t587M_BylxbAL9SQBQlqMDyoycoidydosdlIai-tAw4Js0QU1rLNRr5rv-RekGGI6zH34QB4cwgboMco5MNDAnwz-tg==).
- Patents / IP: 3M historically produces many patents annually (thousands over multi‑year periods), and the company reports climate‑focused R&D programs; however, detailed patent listings specifically for the newest climate tech (CO2 sorbents/green hydrogen catalysts) are not provided in the public summaries returned here [macrotrends](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpCrO-9fCHzn3_IP2r14XP1bD-IzFAXnj5N6O7Ff14F4qSznVp29a6SSd7ZIVxyHjQMGcASDfpYVABq1JoG6gY5FjELqy373PtgvLQROB3kNxL9q3wYs8oYXQPKI6tV1t065HVuHpWxZIRg-Ca9CmezjSH8fERZyMcZEoH6wJLkRInpEdd) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEosLDo-yYo6v9XB2uGTkQvmVimWuQlXbrqBJhKmm-jXKB-Rrz8chocIQ6wt8pkopZwPfHnUJmVg-QmvQizw1GdzoSW9GwExcflsKYW8nQ7FSLe5tuxPZAlhMZ1oD9KwCzn_ZR_YL4LYft5usTMsPN31Oktvl17jr6VdB62Rfpq7BNoeBJot7gdNdgINX-1fGV7iMTa_DzuFRrjg==).

Why 18/25
3M demonstrates a strong innovation engine (proprietary materials science, strategic investments, venture and partnership activity, and climate‑tech pilots). The score reflects real leadership in materials-based climate technologies, but is held back slightly because some initiatives remain partnership-led pilots or early-stage commercialization (e.g., Svante sorbent scale-up) rather than fully proprietary, large-scale 3M-owned commercial deployments with publicly documented patents/volume production metrics in 2024–2025.

Key citations supporting this section: [macrotrends](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpCrO-9fCHzn3_IP2r14XP1bD-IzFAXnj5N6O7Ff14F4qSznVp29a6SSd7ZIVxyHjQMGcASDfpYVABq1JoG6gY5FjELqy373PtgvLQROB3kNxL9q3wYs8oYXQPKI6tV1t065HVuHpWxZIRg-Ca9CmezjSH8fERZyMcZEoH6wJLkRInpEdd) [circularcollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzT9mdzm8yoYZc4ExpF9AQ6FtK4GdKbV_RB0xYEnaepct15N6fo1rd_oMqTmsfEPVBg5dbPxex25KiOCFt9sVrR8XxlFmqGVxzAj42e3tXzY78nHtDj6-nZEkXeJzA2qgaELAqIS2zHAAPTJAepkyv4ADPm_cji-CvjZbUmZxvQTogrsk8XwyLIz32_mxqfg==) [esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHNEbPLZp0dsc_ti4E5gw-7BaMsoi4DFKktRqRKrbuiaoBm2yH6UGZww-vwK98J7igdKdK67jupfTPjwTYmrfUjqdxXWamt9Ewt4CbMnImLo5cFTShpoNJwbbnCsipey0WdC2Pu6oAlvfDbrbFx-BiAbt1StsWKLxw7NRWYiO-IGRfCqSE_T2NssG9P1Di50Oe0hWEuoA==) [svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFNex9LWz7hAoNsCp4baJAK5N8zgQ9ENPWBwi6qnbc_TeEjPD-Gbtn7Ovpci74dWcpy6cmYxGxCjDU5jFvP8LE4XToiPYFtwXqnX3F2Ft9G68VVIgExQrZ4rLb-TurTgIBnNDBdR1WaLqMBcXOT6XpV2c2ethF3UGapIRjhXHNHGgs7Hu8QseL5V6n3GV3gy7vyq8sXKc83bZkrUQ==) [3blmedia](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGTzFA3XCWGMZlGvaFBq6RlsQfRRLRnwyRYzA6XTTNoNmmC9eDVCYu38RENfa6o5o5RLg2aL0oatwesCA7p20telAO7a2SQvyL_6mo_xRYWcJNr7drfRxcIxmqhbaza3CxXrueua5Xf02UE-jPpjK9l8Tvze7i9QG58BnfQOgIcRKWOqCk=) [environmentenergyleader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyxCGkuRYZ9if4m2PAhpZCCwfzuFWxzC-_GUGjKBjPEEkWY7MICwVlNF6uBPRFOrcdMR9hvWiT9nHxjn6Sc94LK0w79QXQnAHU08tKykq-f6LLz4K53Yu4ROBEJnz2fEJTWIuEjVVyb-W6kb78AJtJ8uMP_MBUS9tOt8xkP4DhTJ-wxSrAFsOJtRjwQw16ve5B0Ga3r_nrlYaT-peZYYz_) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEosLDo-yYo6v9XB2uGTkQvmVimWuQlXbrqBJhKmm-jXKB-Rrz8chocIQ6wt8pkopZwPfHnUJmVg-QmvQizw1GdzoSW9GwExcflsKYW8nQ7FSLe5tuxPZAlhMZ1oD9KwCzn_ZR_YL4LYft5usTMsPN31Oktvl17jr6VdB62Rfpq7BNoeBJot7gdNdgINX-1fGV7iMTa_DzuFRrjg==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHR4NqeTMhABBcRbuaaHRehjCH2rkMVEW8No-mjQY_Yj9LYUaZG99Mr39-ubkm49dHlayZNnUo8gG9FmNZXV33d1dY3CxlEAzn7jf0HO-zAomwrI78ggXmDF3Mb_J58NWrJZZMbxE_WAj7gwAESIII0YTGn9CJHetf7_H3eMTs-PZjx0SJ-0rMUl7XN3NwXI5ku7m-3igGUi3YFGvaVSa2uilFVDxs=)

Measurable impact (25 points) — Score: 22/25
Findings and examples (quantified)
- Scope 1 & 2 emissions: 3M reports a 59.1% reduction in scope 1 and 2 GHG emissions since 2019 (as of 2024), ahead of its 2030 interim goals and validated by SBTi for near‑term targets [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpeq3TiNzhZ7rsXhJcd92lhE0jDHpZ7k18pruwRHchQ6UShj7PAGvDMnHKBmq_eJY_ZBNPNfEeRGNDuoVA-HVsVR7cZaEa1LvSgTdRKQgfXI7tiDczH92PVctjUkyL4cVqRthoJdJ4IJE=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQERtT3id3Tx__xCtmUJ0jTgNQnoOh_oi8ss7PkRP4CxRzpXyFIEppYXJhQmNGa9H1mVMx1U1YDOEPcZv8SYlaTw6bnGoOFMWwRc8SquhRMU8HVFBkeRMsEpwL_KB0uPqw9HrLV-kRZKc2Ybv0UXQsIgy2Adg_KSMnZgBXY2M7nXUNsGaV5pxiw=) [smartenergydecisions](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGPJgRqjWEXigkzD119q5mY6V8hLEa_jWaUes7OoMtPH0t_MEQiIxb-S5gwbtv_MXsKacGDkhoZYOTG5Fl8b_JHDvwzqVYMQbsdFtCqsi01HJH_Rm4D6-0TKwdaCrvvG3Lsteqda3WtPGOd07YXyFowPqZxVwRucxesznMDTNDTOgw=).
- Scope 3 progress: reported ~30.7% reduction in scope 3 since 2021, with SBTi-validated targets for further reductions by 2030 [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpeq3TiNzhZ7rsXhJcd92lhE0jDHpZ7k18pruwRHchQ6UShj7PAGvDMnHKBmq_eJY_ZBNPNfEeRGNDuoVA-HVsVR7cZaEa1LvSgTdRKQgfXI7tiDczH92PVctjUkyL4cVqRthoJdJ4IJE=) [net0tracker](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF2sR1kzZz_3zvlz6vAmCFJNlPaXgJ0n9uCkx0ngW8iOz1BrK9GsGiaT8bOPwl1RFNBDufLP7t_tyorGrvFveHv-8yoBzk-MxWLJVXBQmveyUaPQe_g0dHhoSnx-RJCQvW2n8kZnbE=).
- Renewable electricity: achieved 56.2% renewable electricity across global operations (2023) and surpassed earlier intermediate renewable targets [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_tZ7qV6oOyYdAoU1i5qgKjYbd8GOt6GG6guRhyeoDR9p1Vpilwf1m1tAG0uygPGk2cjeFnfPYu6mI7Y2-nnU514ToRlbiSL1TQylK1c_V9ah8GwurzvYbCKOBV-2B5qghpq_JuHbXye5YcQ==) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHiNIo8KbfkmvPZX5P5RiuSpeK6EZq4Fd_wmWdhqe-rhSlQfuxJZSK1Ut6XGTA9HpUC7vfYE4Zb0Gnq-T4hywyKfFalRi3AlBkJxnlO8OhdxxOENIKVejdmUE8-K0fO5XTC47aK1W970JHDQrqiz71ighxGRa8uzEe5TkMx7Q5Wsw==) [cleanenergyministerial](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQESVFPMEEL2juTiArb27s32odJhiTNwZLXxgFAgAA5RNqGfM2kO2BOkVsJojJz8ajOPoh9xNo-y0RZNOIO50JubGb_n7pzMAVx73OY1CzFA25kJHgA8o3HP15ihhjgPoWdjeMzv7Rt6T8fDGTFPHpJnbZvzFZHcstD7yfa424FoBAPmXOzGVgDj-hv1IhoSxeZMnnle42Np).
- Water efficiency: reported ~21.4% reduction in water usage since 2019 and installation of advanced purification at major sites, with a corporate target to reduce water use 25% by 2030 [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpeq3TiNzhZ7rsXhJcd92lhE0jDHpZ7k18pruwRHchQ6UShj7PAGvDMnHKBmq_eJY_ZBNPNfEeRGNDuoVA-HVsVR7cZaEa1LvSgTdRKQgfXI7tiDczH92PVctjUkyL4cVqRthoJdJ4IJE=) [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLOa6zCaiIcCrlH7SmogmVdF3RxQ9HDhjQJFAAu1m8ZBS6JcXTVxtD_FuAw7_G5qvAmGFVPBBbUxcZvbmilPfhsoBvwJZDaEmuwz6fjaSplFUIRMB3VkgL7zHbmcNjfU_cBEn3W2rgDBNpohHayqN4HlTJiMGmboaIJ4ddcUD9w9ACy5GJT_e_FWP_doyvY2hpu2iV60j655gbnVJHkj-GMFaLgaRQrfv2_-ZyOvjI).
- Plastics & waste: 98.2 million lb reduction in virgin fossil‑based plastic use since 2021 and targets to cut virgin plastic use by 125 million lb by 2025; broad progress on zero‑landfill sites and waste diversion [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHiNIo8KbfkmvPZX5P5RiuSpeK6EZq4Fd_wmWdhqe-rhSlQfuxJZSK1Ut6XGTA9HpUC7vfYE4Zb0Gnq-T4hywyKfFalRi3AlBkJxnlO8OhdxxOENIKVejdmUE8-K0fO5XTC47aK1W970JHDQrqiz71ighxGRa8uzEe5TkMx7Q5Wsw==) [able123converting](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEr2j00tNVz29NF9w4v7UJsWV-yuELJEnwq8yI-VDXdQpkgpVpSkn44EdUMpjSQ8y6f3Or4985wG77FthjjG8LcTe_o9hjAgYHjiJbM7QOZbmpeULT35V-0FWCKDpAOnLWx9Bcw-bJOJQReqPGO49W9AmVwpzyxVwbznkQQEaEQHpLmmiav6AzY9lbU-kVXafcC6gsGc0po5DvgQBo54FCtZjrLryM=) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEosLDo-yYo6v9XB2uGTkQvmVimWuQlXbrqBJhKmm-jXKB-Rrz8chocIQ6wt8pkopZwPfHnUJmVg-QmvQizw1GdzoSW9GwExcflsKYW8nQ7FSLe5tuxPZAlhMZ1oD9KwCzn_ZR_YL4LYft5usTMsPN31Oktvl17jr6VdB62Rfpq7BNoeBJot7gdNdgINX-1fGV7iMTa_DzuFRrjg==).
- Customer-side impact: 3M reports that its products enabled customers to avoid nearly 14 million metric tons CO2e in 2023 (product-enabled emissions avoidance) [cloudfront](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGIlDalpWz17n3umhQJOW5SFhY4iZAel4U0ZzZ2IB3t9JUdn9hgEhxQRZqN695gEbTWl_pCheG4Xdx97wcaArq270X7qMEOsoDTu9DyFrrZJVNaGgZJn0xXRSZg0bYLFFN3uuq51edfr32onycUhR9CC9e41xwNiOplmaevJk93vpAvuVxNh17JBbLCjQI1aEBoDAnUUJzCkt1YE0mNBnS175-U-wbMvjORNuTo0n8bkn6l28_eJmlcp2aocPLHfuL4roYrj7HJ24CG6Fgw).

Why 22/25
3M demonstrates major, documented environmental improvements — most notably the >50% reduction in scope 1 & 2 emissions since 2019 and significant gains in renewable electricity and plastic reduction. These meet the threshold for “documented major environmental improvements.” The small deduction reflects remaining challenges in absolute scope‑3 reductions and the fact that certain impact claims (e.g., future CO2 capture volumes from new sorbents) are prospective or collaboration-anchored rather than fully realized yet.

Key citations supporting this section: [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpeq3TiNzhZ7rsXhJcd92lhE0jDHpZ7k18pruwRHchQ6UShj7PAGvDMnHKBmq_eJY_ZBNPNfEeRGNDuoVA-HVsVR7cZaEa1LvSgTdRKQgfXI7tiDczH92PVctjUkyL4cVqRthoJdJ4IJE=) [smartenergydecisions](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGPJgRqjWEXigkzD119q5mY6V8hLEa_jWaUes7OoMtPH0t_MEQiIxb-S5gwbtv_MXsKacGDkhoZYOTG5Fl8b_JHDvwzqVYMQbsdFtCqsi01HJH_Rm4D6-0TKwdaCrvvG3Lsteqda3WtPGOd07YXyFowPqZxVwRucxesznMDTNDTOgw=) [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_tZ7qV6oOyYdAoU1i5qgKjYbd8GOt6GG6guRhyeoDR9p1Vpilwf1m1tAG0uygPGk2cjeFnfPYu6mI7Y2-nnU514ToRlbiSL1TQylK1c_V9ah8GwurzvYbCKOBV-2B5qghpq_JuHbXye5YcQ==) [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLOa6zCaiIcCrlH7SmogmVdF3RxQ9HDhjQJFAAu1m8ZBS6JcXTVxtD_FuAw7_G5qvAmGFVPBBbUxcZvbmilPfhsoBvwJZDaEmuwz6fjaSplFUIRMB3VkgL7zHbmcNjfU_cBEn3W2rgDBNpohHayqN4HlTJiMGmboaIJ4ddcUD9w9ACy5GJT_e_FWP_doyvY2hpu2iV60j655gbnVJHkj-GMFaLgaRQrfv2_-ZyOvjI) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEosLDo-yYo6v9XB2uGTkQvmVimWuQlXbrqBJhKmm-jXKB-Rrz8chocIQ6wt8pkopZwPfHnUJmVg-QmvQizw1GdzoSW9GwExcflsKYW8nQ7FSLe5tuxPZAlhMZ1oD9KwCzn_ZR_YL4LYft5usTMsPN31Oktvl17jr6VdB62Rfpq7BNoeBJot7gdNdgINX-1fGV7iMTa_DzuFRrjg==) [cloudfront](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGIlDalpWz17n3umhQJOW5SFhY4iZAel4U0ZzZ2IB3t9JUdn9hgEhxQRZqN695gEbTWl_pCheG4Xdx97wcaArq270X7qMEOsoDTu9DyFrrZJVNaGgZJn0xXRSZg0bYLFFN3uuq51edfr32onycUhR9CC9e41xwNiOplmaevJk93vpAvuVxNh17JBbLCjQI1aEBoDAnUUJzCkt1YE0mNBnS175-U-wbMvjORNuTo0n8bkn6l28_eJmlcp2aocPLHfuL4roYrj7HJ24CG6Fgw)

Future strategy (15 points) — Score: 13/15
Findings and examples
- Targets & timelines: 3M has clear, timebound targets — 50% reduction in emissions by 2030 (scope 1 & 2), 80% by 2040, carbon neutrality by 2050; 25% water reduction by 2030; plastics reductions by 2025; PFAS exit by end 2025 [esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGjbYX9i4jU9b4pvHyZC4tVpOx8x7Tpwn9TfrI2aEt8671x3VBK8xqWAkHrRONo4WXTaIIxj2ETB8C668UnlqQOVbmN08wMb71RAwe9Stl52Iz1RQah_xPleEf9e9MH_klvdpW9LwO5_uPx5C7w1D6mFP320JVSHSflC9bovD5HEqPqqBpN3cdtjF_kbb6-v1wj10ze) [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGvIxVmji0KnbuxzhHhYFZSU3jj2oCY4JPs113sAfJfaVVYUVgDUmYFZ_env6X34bORgFTlFWJ9ucpnJja3DatQAbgXA-lFXFAL-LQsQRjOnXBKgYC4wb6iCyYxJ5bguF8LKkxCXhsgy_N7OsDkVrSBy5QUUTxSXjIaISC4X9DR9rql7hM0PMHGgir0QvLk0on_zSe2Od-qPP-bqlQ9PN_xXZma-_pZSLL6Fizq-TE=) [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLJpCD_1KfwdICe191Ztl_l4WkG1KfrtsL2oXcWKUuol90GYPktytFZvpbsb5tyLnz_LfoNd23DZAzXCrU9Mi3kKul4JswxJtvQvxwn_igM221ViS0MgqmbSTj0UxE88SMgzi_AXz7PCGgyhVH9XpDfxyUJ2JN-BllUyibEQGwVugRA6kDCIgjsddFFSMLK_3s0RGODCnXUenC) [manufacturingdive](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEkEeheEFOGptqmzNjYL-A6gU9yEnd_snO2gUayY0Jus5W5JWVyqFVpHhXzoJ3unS0H3yEGEJeM0zT8U0ban4r4yRe3_C1nvabnwp-MTDmp1TsozliyTCkw-Z3cn-Qy4DrJUc8Nygv_Zv6xsdoH9TPPLP0GhbwtabHrQirDke_YK88Z9unMOhUd8gOX0okXHJKfTgK-lbzXcqYe0ofgc1W2692jVKlbhQM1SLAt).
- Validation & plans: SBTi validated 3M’s near‑term targets (2024), and 3M has committed to publish a climate transition plan by end of 2025. 3M also committed ~$1B over 20 years to support climate and water goals, and announced R&D/capex plans to boost product innovation 2025–2027 [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQERtT3id3Tx__xCtmUJ0jTgNQnoOh_oi8ss7PkRP4CxRzpXyFIEppYXJhQmNGa9H1mVMx1U1YDOEPcZv8SYlaTw6bnGoOFMWwRc8SquhRMU8HVFBkeRMsEpwL_KB0uPqw9HrLV-kRZKc2Ybv0UXQsIgy2Adg_KSMnZgBXY2M7nXUNsGaV5pxiw=) [greencentury](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFCZSS9tn4MWPJ4HnHhEx9OrIvJ-IAlKzpZoVPy0Zboaicrw4294GxT3EPueAL5CpGnyYmmat9T6N-k1dA6DY5TxMRJi3-zesKSYQXVls2iD9V1dmW3Y7y1MY7mzs2M__ydFQBOvv8iREPwbnZcQ2lVpQfBxmrr4JBeWxq4W6rpqh7s2t6nLC6kfEFO) [circularcollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzT9mdzm8yoYZc4ExpF9AQ6FtK4GdKbV_RB0xYEnaepct15N6fo1rd_oMqTmsfEPVBg5dbPxex25KiOCFt9sVrR8XxlFmqGVxzAj42e3tXzY78nHtDj6-nZEkXeJzA2qgaELAqIS2zHAAPTJAepkyv4ADPm_cji-CvjZbUmZxvQTogrsk8XwyLIz32_mxqfg==).
- Business integration: Sustainability Value Commitments required for every new product and supply‑chain traceability work show integration into product development and procurement [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHiNIo8KbfkmvPZX5P5RiuSpeK6EZq4Fd_wmWdhqe-rhSlQfuxJZSK1Ut6XGTA9HpUC7vfYE4Zb0Gnq-T4hywyKfFalRi3AlBkJxnlO8OhdxxOENIKVejdmUE8-K0fO5XTC47aK1W970JHDQrqiz71ighxGRa8uzEe5TkMx7Q5Wsw==) [able123converting](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEZ5RJ3a30BofiUr7GCnv_JS9jLTR543Gt2_x15iN9eUxzwSmOxPCRMvS1AirRq0BIAe9F-PdfX1jQWu-MvDJ9a376z9QfX8uW55RIE9pPIXVhYQAaALDHJITDLP676HkTqUT9UPNwIHrQ2vMrcvqe3LVlRWpqmofMnH2AoBJCUbBEKfJGlE0kDn1WNoc3DlbM84JIj0ZMgITo6J5LEC0fa9u_s4et0).

Why 13/15
3M’s roadmap is specific and quantified, with public targets, SBTi validation, and an announced climate transition plan. The company’s long-term capital and R&D commitments are explicit. The score is slightly below maximum because some elements (e.g., full climate transition plan publication and scope‑3 pathways at scale) were still in progress as of the latest reports.

Key citations supporting this section: [esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGjbYX9i4jU9b4pvHyZC4tVpOx8x7Tpwn9TfrI2aEt8671x3VBK8xqWAkHrRONo4WXTaIIxj2ETB8C668UnlqQOVbmN08wMb71RAwe9Stl52Iz1RQah_xPleEf9e9MH_klvdpW9LwO5_uPx5C7w1D6mFP320JVSHSflC9bovD5HEqPqqBpN3cdtjF_kbb6-v1wj10ze) [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGvIxVmji0KnbuxzhHhYFZSU3jj2oCY4JPs113sAfJfaVVYUVgDUmYFZ_env6X34bORgFTlFWJ9ucpnJja3DatQAbgXA-lFXFAL-LQsQRjOnXBKgYC4wb6iCyYxJ5bguF8LKkxCXhsgy_N7OsDkVrSBy5QUUTxSXjIaISC4X9DR9rql7hM0PMHGgir0QvLk0on_zSe2Od-qPP-bqlQ9PN_xXZma-_pZSLL6Fizq-TE=) [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQERtT3id3Tx__xCtmUJ0jTgNQnoOh_oi8ss7PkRP4CxRzpXyFIEppYXJhQmNGa9H1mVMx1U1YDOEPcZv8SYlaTw6bnGoOFMWwRc8SquhRMU8HVFBkeRMsEpwL_KB0uPqw9HrLV-kRZKc2Ybv0UXQsIgy2Adg_KSMnZgBXY2M7nXUNsGaV5pxiw=) [circularcollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzT9mdzm8yoYZc4ExpF9AQ6FtK4GdKbV_RB0xYEnaepct15N6fo1rd_oMqTmsfEPVBg5dbPxex25KiOCFt9sVrR8XxlFmqGVxzAj42e3tXzY78nHtDj6-nZEkXeJzA2qgaELAqIS2zHAAPTJAepkyv4ADPm_cji-CvjZbUmZxvQTogrsk8XwyLIz32_mxqfg==) [manufacturingdive](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEkEeheEFOGptqmzNjYL-A6gU9yEnd_snO2gUayY0Jus5W5JWVyqFVpHhXzoJ3unS0H3yEGEJeM0zT8U0ban4r4yRe3_C1nvabnwp-MTDmp1TsozliyTCkw-Z3cn-Qy4DrJUc8Nygv_Zv6xsdoH9TPPLP0GhbwtabHrQirDke_YK88Z9unMOhUd8gOX0okXHJKfTgK-lbzXcqYe0ofgc1W2692jVKlbhQM1SLAt)

Industry position (10 points) — Score: 8/10
Findings and comparisons
- Relative strengths: 3M’s pace of Scope 1 & 2 emissions reductions (>50% since 2019), achievement of >50% renewable electricity (2023), extensive material/circularity programs, and visible climate‑tech partnerships position it among the top performers in the diversified industrial/materials sector [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpeq3TiNzhZ7rsXhJcd92lhE0jDHpZ7k18pruwRHchQ6UShj7PAGvDMnHKBmq_eJY_ZBNPNfEeRGNDuoVA-HVsVR7cZaEa1LvSgTdRKQgfXI7tiDczH92PVctjUkyL4cVqRthoJdJ4IJE=) [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_tZ7qV6oOyYdAoU1i5qgKjYbd8GOt6GG6guRhyeoDR9p1Vpilwf1m1tAG0uygPGk2cjeFnfPYu6mI7Y2-nnU514ToRlbiSL1TQylK1c_V9ah8GwurzvYbCKOBV-2B5qghpq_JuHbXye5YcQ==).
- Peer benchmarking: Peers (DuPont, Honeywell, Emerson, etc.) have strong programs too — some report comparable or faster achievements for specific metrics (e.g., DuPont and Emerson reporting deep reductions or early achievement of 2030 targets in some categories). 3M ranks among the leaders but is not universally the single top performer across every metric [net0tracker](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF2sR1kzZz_3zvlz6vAmCFJNlPaXgJ0n9uCkx0ngW8iOz1BrK9GsGiaT8bOPwl1RFNBDufLP7t_tyorGrvFveHv-8yoBzk-MxWLJVXBQmveyUaPQe_g0dHhoSnx-RJCQvW2n8kZnbE=) [smartenergydecisions](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGPJgRqjWEXigkzD119q5mY6V8hLEa_jWaUes7OoMtPH0t_MEQiIxb-S5gwbtv_MXsKacGDkhoZYOTG5Fl8b_JHDvwzqVYMQbsdFtCqsi01HJH_Rm4D6-0TKwdaCrvvG3Lsteqda3WtPGOd07YXyFowPqZxVwRucxesznMDTNDTOgw=) [esgnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLlcx0zYUj92qINJCwksBK7-J03BpxNwgE_WDIoaCRA91ShZ_YrINH0lW_6YDP8J1gCzBKWzq3YZxQLj0MRLGlgebRqZlbAOSeCevHHtAj7A5QiddiswutLqOB8EuW-hFiN59o8W13pzZZQ0AJuEEEEwqzjy-BQmFnDIY=).
- Differentiators: 3M’s materials science footprint (high patenting, manufacturing scale), product-enabled customer emissions avoidance, and specific climate‑tech partnerships (Svante) are differentiators versus many peers [macrotrends](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpCrO-9fCHzn3_IP2r14XP1bD-IzFAXnj5N6O7Ff14F4qSznVp29a6SSd7ZIVxyHjQMGcASDfpYVABq1JoG6gY5FjELqy373PtgvLQROB3kNxL9q3wYs8oYXQPKI6tV1t065HVuHpWxZIRg-Ca9CmezjSH8fERZyMcZEoH6wJLkRInpEdd) [svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFNex9LWz7hAoNsCp4baJAK5N8zgQ9ENPWBwi6qnbc_TeEjPD-Gbtn7Ovpci74dWcpy6cmYxGxCjDU5jFvP8LE4XToiPYFtwXqnX3F2Ft9G68VVIgExQrZ4rLb-TurTgIBnNDBdR1WaLqMBcXOT6XpV2c2ethF3UGapIRjhXHNHGgs7Hu8QseL5V6n3GV3gy7vyq8sXKc83bZkrUQ==) [cloudfront](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGIlDalpWz17n3umhQJOW5SFhY4iZAel4U0ZzZ2IB3t9JUdn9hgEhxQRZqN695gEbTWl_pCheG4Xdx97wcaArq270X7qMEOsoDTu9DyFrrZJVNaGgZJn0xXRSZg0bYLFFN3uuq51edfr32onycUhR9CC9e41xwNiOplmaevJk93vpAvuVxNh17JBbLCjQI1aEBoDAnUUJzCkt1YE0mNBnS175-U-wbMvjORNuTo0n8bkn6l28_eJmlcp2aocPLHfuL4roYrj7HJ24CG6Fgw).

Why 8/10
3M is clearly among the top performers for sustainable technology adoption within its sector — strong in emissions reduction pace, renewables, and material‑science applied climate tech — but a small deduction reflects the very strong, sometimes faster claims from select peers in discrete areas and that some 3M innovations are still in partnership/pilot phases.

Key citations supporting this section: [3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpeq3TiNzhZ7rsXhJcd92lhE0jDHpZ7k18pruwRHchQ6UShj7PAGvDMnHKBmq_eJY_ZBNPNfEeRGNDuoVA-HVsVR7cZaEa1LvSgTdRKQgfXI7tiDczH92PVctjUkyL4cVqRthoJdJ4IJE=) [net0tracker](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF2sR1kzZz_3zvlz6vAmCFJNlPaXgJ0n9uCkx0ngW8iOz1BrK9GsGiaT8bOPwl1RFNBDufLP7t_tyorGrvFveHv-8yoBzk-MxWLJVXBQmveyUaPQe_g0dHhoSnx-RJCQvW2n8kZnbE=) [smartenergydecisions](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGPJgRqjWEXigkzD119q5mY6V8hLEa_jWaUes7OoMtPH0t_MEQiIxb-S5gwbtv_MXsKacGDkhoZYOTG5Fl8b_JHDvwzqVYMQbsdFtCqsi01HJH_Rm4D6-0TKwdaCrvvG3Lsteqda3WtPGOd07YXyFowPqZxVwRucxesznMDTNDTOgw=) [svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFNex9LWz7hAoNsCp4baJAK5N8zgQ9ENPWBwi6qnbc_TeEjPD-Gbtn7Ovpci74dWcpy6cmYxGxCjDU5jFvP8LE4XToiPYFtwXqnX3F2Ft9G68VVIgExQrZ4rLb-TurTgIBnNDBdR1WaLqMBcXOT6XpV2c2ethF3UGapIRjhXHNHGgs7Hu8QseL5V6n3GV3gy7vyq8sXKc83bZkrUQ==)

Final (calculation)
Technology Adoption: 22/25  
Innovation Leadership: 18/25  
Measurable Impact: 22/25  
Future Strategy: 13/15  
Industry Position: 8/10

Total = 22 + 18 + 22 + 13 + 8 = 83/100

Concise wrap-up and implications
- 3M has moved beyond compliance toward true sustainable‑technology deployment at scale (renewables, water purification, circular materials) and is investing in next‑gen climate tech (CO2 sorbents, electrolyzer catalysts). The company’s metrics (scope 1 & 2 reductions >50%, >56% renewable electricity, large plastic reductions, water efficiency gains) demonstrate measurable outcomes and provide credible basis for strong scores. See SBTi validation and public targets for evidence of credible future alignment.
- Areas to watch: commercialization scale and independent verification of novel carbon capture deployments, further reduction pathways and transparency for scope‑3 emissions, and sustained year‑over‑year R&D spend focused on climate technologies to convert pilots into owned, high‑volume products.

All cited sources used in the summaries (click label to view):
[wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFS_G2myAfxSw2BiSzUSLneKM6EWDLSKyQE5YWu_VpQUh9krbuBzK2ZlbDhKjNE8qGI1pTt4ROeUxic7MI-kozj-K_Rfhe3dJUWdfSWq8T0iFgEeBjKh2j65yK8ctG5d_xXrh-B2XulfvjMZw6_45_o1d3uIX7vRmd0ZnlNzY6OfZU-31_FkKLx9CWi-13egY3zZGmJYD-kN70BU1NG6IO-LVY05fSqsIsa3_5fIlU_)  
[esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE3avsXWgsIkGQA-0qA9TMt1co6c0BO5UO-DMXCmv_FeRQ1gcLK6vYSVkg7lyedOCQXfT4o0mUrsDc-Ns8q4U1W0tXF77oDO2g7mGcYmY9bpUyjdceulRGIaxyJJyg2y4Y6RKQr-YrAte6KoaGdy49CcCB6lhssNgzpPg0QrAMGdfccK8f5VVJLn9tJjtNDFWIocmWikw==)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFxgm1dfJ4GGvdOxGy4Pj-OvMDeJnIqhIshaaplk5-xsB1-YXddJbNXVZWEejPVDhigoiinMWITmmkKrv9WMY9TCaD2N5TZ1gjuKgtqtwygFQd9nCtr1bSFDtOlQC_RKYz6Ou5TNt9JgAw=)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEloVoxHVUEogl4xMZziOCyDEEQmoTy09ighkK8q1xciX2hQ6-igIlZfM66Pfqr-jJ6R6pLo1Kt_3YvWfCw3mw5x6eSrguKrHqudInyWnGiEN_byfbdHlRbjIAgcGFqjRSjW2w6TnCm1IVc9na8J2BK_4anONc6D4hU9T_F_HJdxxz_QwORLdCT)  
[sustainable-energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG2C5ZQF--T2SKOChFdhsIT-h89fzaJAVLcBp6gvruI_3PJr_V7PNvFWlVe3rVf8LNqKGahmor4ZY8yeYZkLjTDh3zzLXq3e7InAz7kSUOfNqVJ-p7gyNF8Xn3zBNRw9GMAdGyVc2Lki91qaJRJWW5N3CcJlWWU1n0xKZlts8vr8BHI_LSnDla6uc3O6qz6ggp-IxKPOVktp9f6-VmEkKUYzOBsyPpzJA==)  
[scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGnjYu9fGR2YKMKlG5ISIjQ8LqnDr3MV6PQIIpewkFNLxRXB5N4fxuyUFbb1mmJHFAvsrnyRUoQooaVwiTs1cO4c2WhsW7Tp4Qjs9LqEapBRgh62-j0PHUe7dhEtxRdKIguz4w3pe3X3uwxLO8=)  
[csrwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHclSbxoDrXr0Pzhsw6AiWH-niS2hyfA4MDR45All9Yuw7nwBiu7Autfq1nItWncYisbmmDYAJoEE7HF-pyXQIUtp4ccrG2Qy7pC4GIeBl8WJbrNVi-qzDlqabBAfaRsxRzaQuq7r6yZO_ozKe0J4EcCHTTU_TX80qBR4cF1TwrVsqYezA2wabYrSdRX8Sf0mUBVqVmda1dZDEsygMNEqs=)  
[thecsrjournal](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFYSSG2r_GMc2_U0P2It0SOXUmBhF1dDn1TuFf-YOhL_ZbyR-TBUlk4SCXLAOlEFpDyrPL3i7JNmVgKPC3RmzQqYyV-jhlv_NgO_QeK6rzUXOGW120Z_9iF1w9deCI-LaD3OaogtCbjANJfqMoxosgDS9NBOEEXQvA=)  
[researchpublish](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFF2aXj7t88uymzKM_z5WyX7MwwVJgXNMnffCmAhlGON1GKUT6JsDSZeHpUyButymDwWNUEIQJSCKBbUQHEJ80z6c7hRiPUfvgoWAzuYxy5_7wDHnpNqeMcEAMqx6N84dz10zvocdFpJoEkFXxeY-LJcdxHzJynGUGeywu8QvIEn-QXDAp9aA_99K4ZkjCkWhaynOkxqo9JMtCR-yTXVQ==)  
[greencentury](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGwPjrBNtEMhvT-fimOvAzWQWo5zLy4OtCyGTbcaLUol-HG8GBgPjiBuhj1_xK74fSH19nkygBnzGEm8xTnqfz4AMt62xaAC-g7sItGmNRFb_5D4aOYZlS3I5pdk1_Tlkx1hjAtr7y21FAUMX2JLAuj4ndULM2rPQw6Hz9hp-ondrz0kEbbbWiWOhJzYg==)  
[sprichards](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEW-nYZeiYi9aEgJkUOskE3YSHgjqVQ8XUre5SCebx2_JzdzMCu1tkoOVYVKF7W4moNVOMU080fD7M_LAs1dm5XR2F2TKzeF7nWfSBqGGj0xkLYR7AN_TfP8nhsT9baH7tZLVLVMVY7mqXk_mcozQsJuFr7sa1v-PlMwzrZwP72g7pJOIGmw8yzb8gV8HTbHGPPFWzo2z6AsRU9gd-LlIbv)  
[responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG4kUZQeAp03WDza4PBcJflXLf02-cmDmjG8k3iQ5Ore7UHvnNGZvSU6wwg1bbTWFo_95p8uyUBShiM4b7PlUwdvxzEaxIQTwu7ctCsaKamvMkK2c8FQ5ihdmhry5pffefNSSY9rtF6_TnwON_i_QD2P0XXeri7HeZIo3ekN8iTI1aobti7QfC5dOAdIYizUuQaFoNVR_dY-j6-dQ==)  
[able123converting](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEZ5RJ3a30BofiUr7GCnv_JS9jLTR543Gt2_x15iN9eUxzwSmOxPCRMvS1AirRq0BIAe9F-PdfX1jQWu-MvDJ9a376z9QfX8uW55RIE9pPIXVhYQAaALDHJITDLP676HkTqUT9UPNwIHrQ2vMrcvqe3LVlRWpqmofMnH2AoBJCUbBEKfJGlE0kDn1WNoc3DlbM84JIj0ZMgITo6J5LEC0fa9u_s4et0)  
[remediation-technology](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE8EEWH_IzGJbou01debNxMryDoMx-RJdmjSPutFxMsC1Gwlo0wVijbSxy-sAty04mJ3_tpT5xQLKQKvOO5YTv7_lYDiMyJ04qtpvNj5VXsDm3aydyp_iV2hG2pT6ETfl1fH4Xndr1POIjZF1TvPALmcubGYuIUDym1LOSpPb2-Q7MihW6Ikk0ZrmzzSKs9hI97wahRXE5ei82eEA==)  
[trellis](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFEooTQqeJeaZpivQPAl3KCIThPpbdnAeNlYKH7gX6bxD2UUTRJglNcl-6GoiRUHR74jLXOqglJXJps3RzLAaYr4fiQYu2H7n--GvKRewtebVlLPqtIVQ3bk0tJfjZeBfbd3PqEuQYYbB0biTF2hTmyUFROa4aZx0OyyyWmuHws0hBmZsET)  
[investigatemidwest](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQErEpvygtm0iRuGWN1akPvi-569pFfXoQTFf2jeMT8b7m8vFegam-A7XWrmv4hkGPHwvOCquleeMdFevYL2zOb0wplv2IJlz3fscWHaUHc_xuyPOoNO_YSAtrgpd9AFruaipwwIRNJRi8IZuijinu3hE8EhoiCaU57WgL7dHdDc3ae5Y16LJkb8x3VDVdRqDLXHGh6_5QbCfWUOOyzkiQVy)  
[csrwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFvpM6ErmGulsAHWbwVDtDYK9To7I1gqchJPL9mIWw6-sofkIkZiOhkjJj1VFaqesJWb8KpD0Ibh4wruZQCWp5b203gI89K4FUVatAU_e4KR8HeR8M0UUfRHys4FRg-_2Yu7NfLP1S_yqSWFfL2QbwH6DWq1lcsKhyPyhib7uvKqMOks5ESlnoiY_rhOenpLpNohlh5IU7IHog=)  
[inkworldmagazine](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHclnC93oyt17kaJGazFYpiofFCX0NFMiJtLiSWJaiXUtrVw857x_GhTLkKlqSdCpSm-9256mb8PADijVRI6UuwJGo9wE74ZIBJ3ibTnepURSrnJbFaV2NIylsA10myMP3f8Acsnhkg0VbvMjY63zylDGcN7MFJ8wrAo1mxIbKCzItQFgI-M4-7e6C292maqT79QRn_)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEy1dTvbtvBs9a9XwU5XmakI4P9V7v6Qsb2Kt87s7B-FFJ1OgmbkeMdcry4ItqNjie66O00Ia92bD010Kkh3d4LkY8pgSRmZ-E9YfPhoPxmKTOWuqhiOjm9DOJ6uo6aC9NHxv0QbE_7nA==)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQERtT3id3Tx__xCtmUJ0jTgNQnoOh_oi8ss7PkRP4CxRzpXyFIEppYXJhQmNGa9H1mVMx1U1YDOEPcZv8SYlaTw6bnGoOFMWwRc8SquhRMU8HVFBkeRMsEpwL_KB0uPqw9HrLV-kRZKc2Ybv0UXQsIgy2Adg_KSMnZgBXY2M7nXUNsGaV5pxiw=)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGghQNBxvQPFEqH05ufEfUY3TEy3qnmZehsV77HY_B9ftGM-yUyQqsT6ySAaf55AQR-J-PHMS9jBlNd3jal12bUFqIcdV6x9JkvZlmcsLN1G8jyyhvDcI_SdnWPbQU8D7GvDuSZcSi0rRYuMB7VznJ1o62DkUIOzqIR_TPDwH6Vl3Xu9wpYFaA2IN986HKIv10DEo08)  
[esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGjbYX9i4jU9b4pvHyZC4tVpOx8x7Tpwn9TfrI2aEt8671x3VBK8xqWAkHrRONo4WXTaIIxj2ETB8C668UnlqQOVbmN08wMb71RAwe9Stl52Iz1RQah_xPleEf9e9MH_klvdpW9LwO5_uPx5C7w1D6mFP320JVSHSflC9bovD5HEqPqqBpN3cdtjF_kbb6-v1wj10ze)  
[wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGvIxVmji0KnbuxzhHhYFZSU3jj2oCY4JPs113sAfJfaVVYUVgDUmYFZ_env6X34bORgFTlFWJ9ucpnJja3DatQAbgXA-lFXFAL-LQsQRjOnXBKgYC4wb6iCyYxJ5bguF8LKkxCXhsgy_N7OsDkVrSBy5QUUTxSXjIaISC4X9DR9rql7hM0PMHGgir0QvLk0on_zSe2Od-qPP-bqlQ9PN_xXZma-_pZSLL6Fizq-TE=)  
[sustainable-energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHhGXbcKvMIG_JuRYbEGn3syA5yLJQ4iBnwF91QbSdW_3Vz19nxJAbSoeBap-xLsHdXuDizlLBwPgahntF76zy5uH6ksfk1ODlPbHfH53fAdGB29zA8jFIFpjxr7yzNIYZDfuxKI8d6WFojTKbEoqMDfGlA0C-0FqDDR58NpRY33T-P437JIWuED-6-YzCQLQ3QIztcywilbNKOjU8rnu2qpaKx9_qp)  
[moneycare](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHxG3q34cRdhgiNTOaD509eKlwvFohIe9CBZxDs_SxAif7Pw4H2eCUDvIEQQzJkPFsZXXN-SgkWXMw616fl53Yv1UbXUenPg4QYe53b0jdiwIVmp6LHrzF-ZIlrvXyO9iyBgdbQLg55cZY6lP1L0nw=)  
[mediaroom](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGRxpd3LdoZ1TAaa16YLpnUREgt8E7m3tGuLKlXcbdCHem7oVWPe00A7DfHCjmvoXd7e8rHpz7QL6ddwlPve_9CyD9kQ-z5V7SzZccRL2LEEPCN8Yvt9Jq8ZQqHOGemfEPSzfWaA4UhcB0fs68w0wokWC0bxgHcxWrVY_-Rk_LEhIymqrvz)  
[responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH0c4hHIOTsjNfZyHRHNis4hKlQVl9AcT_QbhzaqgVwcRD8SWvF7Y9rAYfs0r1Ely13L9M9u-WylwTLZSHUSxeAQDNMN6WB33hAR2WFlWGF1S4lhbhxoulQKEcmbcjheaPjN_K2o3h8L2awN8skmzG6AfU3u8gQeIm1Mm4PiYGxMADtkAEI1Fks9pKs9pfaC-crRdEs4SY0lkwd)  
[thecsrjournal](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHWyejYjpCe9_nALfkX9MYdu1cYFIL3RzMiUIAmGgtvi4-PBrOL4P5kVyI2-ZnmNU0SfX6bQ4oLLOOErc-uY-CYiBIjiImUwh9IxYZVvRfjRJM6giV5NRA5qPXS0rgh49T43HSGqYjwDeKbJk63i5y5X_N8yWOkOA==)  
[scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFuNZdw-t-zXxJZ6r52OM0Meu9F4SqB2ne3E4D500KTI0EFHkfYB9805kN_lLGxI3304i8YHHcRfmi0z_HVX7gnWDzBGZ27vXVVGK1ekhI9g5cbn2nbcgvtc-W4k6qlzBh2Ez2wqDw-0wGJZQ==)  
[sec](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHfU8jgq0E58dpA-1BhU5nQy0TprM34EXyU9pO7sF-VwUjouKQ-DEU2bC_JXq4rIFxYg7WuE4GmgnF-Tea0q8zeTkdt4XVgT-o50Rm2VfLronyNCsQ39wduQREIe5keqmhcol6KSLYAuW1-LBRRqfHppvfu5jOAbhHs24H1cRw27Xb2PC40tlqWoQHZSsM=)  
[able123converting](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEr2j00tNVz29NF9w4v7UJsWV-yuELJEnwq8yI-VDXdQpkgpVpSkn44EdUMpjSQ8y6f3Or4985wG77FthjjG8LcTe_o9hjAgYHjiJbM7QOZbmpeULT35V-0FWCKDpAOnLWx9Bcw-bJOJQReqPGO49W9AmVwpzyxVwbznkQQEaEQHpLmmiav6AzY9lbU-kVXafcC6gsGc0po5DvgQBo54FCtZjrLryM=)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE_razOYlFFLLtsMaFmsjUN36kiSsxwjbP6tC1-yYhMlxe64CoQ0FwHW8u4WCsmtyMv3CDBlj8HCGaAwLmW1DwTDRYsGrdYwX5HnAiSK9FMBVZngfSlsz_4N7xcPh6AZm3EIUjFEu50L5D86nb67_aM-0i2jGqNidaUK84LEUoPVUgbAc7w2ir51rUja7vCBuL21Bh9d2TS)  
[cleanenergyministerial](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQESVFPMEEL2juTiArb27s32odJhiTNwZLXxgFAgAA5RNqGfM2kO2BOkVsJojJz8ajOPoh9xNo-y0RZNOIO50JubGb_n7pzMAVx73OY1CzFA25kJHgA8o3HP15ihhjgPoWdjeMzv7Rt6T8fDGTFPHpJnbZvzFZHcstD7yfa424FoBAPmXOzGVgDj-hv1IhoSxeZMnnle42Np)  
[manufacturingdive](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEkEeheEFOGptqmzNjYL-A6gU9yEnd_snO2gUayY0Jus5W5JWVyqFVpHhXzoJ3unS0H3yEGEJeM0zT8U0ban4r4yRe3_C1nvabnwp-MTDmp1TsozliyTCkw-Z3cn-Qy4DrJUc8Nygv_Zv6xsdoH9TPPLP0GhbwtabHrQirDke_YK88Z9unMOhUd8gOX0okXHJKfTgK-lbzXcqYe0ofgc1W2692jVKlbhQM1SLAt)  
[trellis](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGKUQm3uCNZvnbxRXIK3nzmzVAx68CYRNvOS6OAkyBZX5wEW2DwXo9aCl0SVei76QfkplI2hje3UZBGjcrDHxxMSbq2ritV8sfYQduN0Sr9Xa5rJGrMmWWzxTqNH8xXim-6X5br__Gz4SsLYoPYUyNiKvDKH3AytDTMzNC4q8hqmCatdOc=)  
[investigatemidwest](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH0AvG5morZKMDw3gKf6ZSi4KCHGNC8HhYOFIB2kX0Mu3cDzAf24zO4T6zeZJaOpeJfte3GmlK80idWHvWM8ptjgpVw4CbO88nP3N-KFH-39_AX-4HHmLFOJr4UjZn09kqzBY4d-ElMJpyU9oMasfoRg31x8Iur-dg-xJbp1QOZTDQmB3TWWf-MjcS4or7V035Kdh98o5BhF--GtgjuQ5I=)  
[net0tracker](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFWv7KbKGQ4wP5CwoItmEpbDUFtb6jwE3cx19RGA-ILOoepwBg0bK8FYGXG6I-cs2wdJpPexPAmlxsJUcGDGdzn-jbAHMCO3c9P7cRmq4hoUeMZXKBARFWTCUpJ8xQYRcw4RVfz85nRdnyQHnCabC1JX366VxvGLxFrYK0o68mfQ9uNSw==)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHQsnSUWn5ZiS1ly87NEt3fzdYFPht9Dwc4_RdK9z9OCnzqX_K1V4wKue_tgJHFC2k39gbGxE4TopIEuKbP9WKU1n1CssPrHl1CWSYC49GF7K_f2kZLke3W7GtK6M9KD-mQHyLU5WuJ5Oc=)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFVzVcFanZzCKpYblXZTctcRYwwsFdsrERqDHDBm5lpcWNzWAgQMhTDpiczrMI6U7Hmd3nT6aRSxtKvrWVs3MepvFJqXQb9rWsg7GNLBQl4L4ncg3A1SgGzUj22y-iF8Hb27ztUp7h73eL_9KFFT6lQHSMeVDbbSiT-9qK5DKIlVBCobUNetXEl)  
[sprichards](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFkodDInmgkTEhQz2JawJ_rcFpMEiL8D3KN5XPkx1v_nYxazRzv02bev9XmpVBPcItKOeNhAUavd_IHfhYLlg38XM-T7vc_2FWYKS922nS6Csj9WGbsuxyLcmpRvqeVaLhAqyyAvBLQ_2G_hnbGBCEqcJj8vQis1N7ANlPED1_pL_ARGXXOJNstoJPcb8hrDRI=)  
[constructionweekonline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHdxPVBxTAbnwD-cFd8svsSGcXcYJlXRi-dfcf5bDojnJhVdiJlSajSu_k8EHZ-J1PWWLHSyyQG37OriXIwv3F4Y-JOmViKTjzzUySmyuMoe4D6eqQwhr3T0xe6WpXhVcmVLsLGWZ8d4oKAgB802hs0-IELiZ2wjlPNs6C6mIn8c6sMziLapd9FsT76X1wiKekOOmIr_K0z76YynxkJ83a6Sb4Dx6yg39470OflfszVfCn7)  
[scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHSANXsTSDj-foZATt5N5HhsAT7CiY13aJBnG7dw2IBDARmgWHWxD-GFp7VXE6jzh4NJMKNCs2gDklOJXfjMfubBT-UAjXzxiKKJvlerpDK7cCnMm7vrkg61r1ff6Rai4_c3a3FF64usf9__fY=)  
[3mcanada](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEgohU5v3E7Q0s1VFJNlz7KY6oIkjGE_TOUsLhO0ebQA-H3WjLf9bU6cHEA01IOacQuTMArGCLn9fdlTh0kdK5Nfrwxd4aGsTGaQQ5sSBGdSmcGY_h7eDOdpV_D7KbD2DiOK32fFN-OdFeLHkYxoDbCMJ1syCgmkrHz8FQNZkxbbZ05F3zDL9-BO-4pcIlIWo78EVtc7xwm8jlR0LkuNac-D7Jboxn5XgUlN3GHQPN1527IjMsutb-uHBQ01ferbhYzRGq51KHttlQRiYH_dzax-06cvQrJRtN2nlFV)  
[wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGEkL3WaOrJZoPZTl-hQ7nxvzh-etga9Ue4wbn8HrPPKpwYSJfT_BAHrLXZwWHIpeSrn6doQZPuHj_8L8sSrTnGTj47vSa3tzXm2hSCaTMkvQjlppUVkcA4YArXNd8IHqITmoR4rpi1RMt6Qxz4tfN1sEbo3ieqjN_zpRqW6YtSWsKolUMbQIblTbr92yu0GQ2wGbJTqGnt0c1SaqhlNmFU8-pXrRVHQOZP2o89be1N)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHvW3UFJKg36cGKQs7Fp6dx9I6XM_EXmXIq8W4cTElAKEQuLFg5ihfeHlcKcdYxhMMZQz--ON7-Oyq4l7Zcd2KFpm6_8ekO6uAh_r1dvoBx7MkPz1tg7zqLS5FvqCBZ9eAA2gckHydpdM80ULtlhkWhj1chq6kPHDAydfp383ylgvwE7j6TAZM4)  
[inkworldmagazine](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGsWe0-Kki3zm9KvoHnsRQSkPce5EuWBEBhdz04igkHzLoj4CRlbJ22SlL6u3-Rbhps-_6OSgbKk-MUMP0WZzeMbbD0c9Qkwgy4-V755pWZXK_wau8hkLMxXTyrN2YhZZM1vHjHDebat3Sg8jTdEYnVa1qvtRvThYX_IgFdnSg_LIXO7AGLGuU3bEEnoftwB1IWkJN1zccMd2z7LcN8DfMdmIRQ7FwcF-h6yyIGBiRscG7A_yJNxr_TYsoNDQ==)  
[energyindustryreview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEFBzxnyJoAANO3x90jmPS66FILplDKPDH4Cey2aAI638epAVVSDIeL7S9APcHHwwQrv8bSnj5bnI7dnLIFRYSe_XCxDDtuV5gZ60lz9PB-Dtcd0z0CMuQiNtLRGYwYEWBjtWGUn2-tK0N6Qozk87_qZcre2sqcVcU5qPOBpdfMm6D0PfDkyg4fOlmMtEwr8C049_SDSx0jtKCM5PzzAoU=)  
[esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGJ9riwRUU9H9xpQFixZm7wpcF1j0DV6zyFk80PkjnI4QaW4nlxWnNdGP6zFcWVTqFmKIEM7sD5hohTxB9eXyAhlk3pMev5BNoNRzLd98ogvHpN34Zag8N7hgrDJ7OQ1U5Nl_tHYSY7XpyHWSbwHR2BFOZGiedeVc6Qs9wW3omOQM4qFUm9Q3le4234CHC0IZDLvHfI6w==)  
[sustainabilitymag](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH1a7Z02HY4xvBzxM_xVn1QUFSxDn-u__gT1AJX0SJ-IwNxdrqCGugLazCOmI5DiMlRDUHtYo6Ui1I76v6mFqTa-S9Jeb16GAz4jKcODaqTlZULU0aA3ebNLkHSQOQoOAY_8yPbUODv3lneBiT30p-mkSlVv0uQkbIPWgkCjIvIom7xRrfNKt5w6G-fSzBZ774joIwl_g==)  
[carboncredits](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHx3WGgQ1BPy8nP0aCgjLDrRFfZ1uxGsAguTeGuKyEWrBZiyoOATJT9Lify5U0Mb0knbICzntm1iutCd3vtLKsV-fmxdd_ktjk_QZ4-hkvDJdncsI9VRH9oAfCtsV1_FtZbzvWdlMQt1dGUKwZaeJdsFO7m8ho8wZpREEXrcY2sdjrjxyt4AXd1qaIPU6EM86Pa5hXSbjtFRBRarDY=)  
[inddist](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE5fRM7ANdbrKOavxS20GerrGOnIWvVS2uDb1UOzs1jASqA3dCp3XI2BwE32_VS1jN3CozWq4nLH2RSLbypcOqYDy_GM0oL8ZwkvsIa7N75c0jne1UJI1Z-13isdKJd90H1f4-pXGtPVSeleaS-kNXrtsl_-FLTx7vzXocvndPp-eYFl_xRmzkLSMvHaWYkao_Q9qbdWsCyfXQ7IIo2USM=)  
[svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG2ulzRbdGnhF2-19zrppybobjZacJHfHvvY8KN4D9uOmL5jJG1krvPiOr_-DktKu4_AabvSBjBZ9u9i7YlwxnkvJySUhaUWzmHpfiLqYZ89BQpVd3o96apNYtDvF12zGrXbIvvY4jiUQnIY7Xm2Pk018jpeqC6EutgQUugaRIKyUK12dR_znGiAdIsdqG1uSeAtupTycLWqsGO)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHiNIo8KbfkmvPZX5P5RiuSpeK6EZq4Fd_wmWdhqe-rhSlQfuxJZSK1Ut6XGTA9HpUC7vfYE4Zb0Gnq-T4hywyKfFalRi3AlBkJxnlO8OhdxxOENIKVejdmUE8-K0fO5XTC47aK1W970JHDQrqiz71ighxGRa8uzEe5TkMx7Q5Wsw==)  
[3blmedia](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGTzFA3XCWGMZlGvaFBq6RlsQfRRLRnwyRYzA6XTTNoNmmC9eDVCYu38RENfa6o5o5RLg2aL0oatwesCA7p20telAO7a2SQvyL_6mo_xRYWcJNr7drfRxcIxmqhbaza3CxXrueua5Xf02UE-jPpjK9l8Tvze7i9QG58BnfQOgIcRKWOqCk=)  
[environmentenergyleader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyxCGkuRYZ9if4m2PAhpZCCwfzuFWxzC-_GUGjKBjPEEkWY7MICwVlNF6uBPRFOrcdMR9hvWiT9nHxjn6Sc94LK0w79QXQnAHU08tKykq-f6LLz4K53Yu4ROBEJnz2fEJTWIuEjVVyb-W6kb78AJtJ8uMP_MBUS9tOt8xkP4DhTJ-wxSrAFsOJtRjwQw16ve5B0Ga3r_nrlYaT-peZYYz_)  
[responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLJpCD_1KfwdICe191Ztl_l4WkG1KfrtsL2oXcWKUuol90GYPktytFZvpbsb5tyLnz_LfoNd23DZAzXCrU9Mi3kKul4JswxJtvQvxwn_igM221ViS0MgqmbSTj0UxE88SMgzi_AXz7PCGgyhVH9XpDfxyUJ2JN-BllUyibEQGwVugRA6kDCIgjsddFFSMLK_3s0RGODCnXUenC)  
[sustainable-energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGlsOUgaFgb88gD6WC8wF38zGPfdb6ahnRG_Mk8dEYZFC_I3faR1x_4_PCXJa3xdK6Di_Qc0Sn3AuWHA5Qpl2SvURaObIx_IwvKd2OHHEbYxkFrCf6-yJH6KdORA7X1IkmcIAkjoU2Eqll0Ilum8dcfZtV_Lo5RMfq4-pBQLyFe6sEOgJM3Wvo0jUnfEdyt6nQFOvGy5HMwEz5fSNk0zr1czUJV3k1G)  
[researchpublish](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH441B-v6VHnLoVCfJyGE30fFPXXJBatwdcvXtgiyNXN-7ZQkEc0jNaHCGhvBACrRpE51r02hNhFNwkt7P7ana6MgQ50br08egENmGUiTKxB5bAvt0IGvkaUy-9VUI_ZUOCjhrQfpY9tFGl-ekvG5-Zd9GodOmjd5L2vPGyx1ov4U6LxfdHYhJeQ_XjT-Z3DbRR1t328HC8P1KXhglb)  
[scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF_tZ7qV6oOyYdAoU1i5qgKjYbd8GOt6GG6guRhyeoDR9p1Vpilwf1m1tAG0uygPGk2cjeFnfPYu6mI7Y2-nnU514ToRlbiSL1TQylK1c_V9ah8GwurzvYbCKOBV-2B5qghpq_JuHbXye5YcQ==)  
[prnewswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG0poW4F6ik2hzAdozUl1b0mb1U7IK1MjYw8wlk0LJzMMFeu7O0uiZ_tX3euBMKp1DZI9V4L4tDqNKTbKWO8Jbw1HHc7SdxlmtBIDtw9-jiXIKF294Z1vN7F2JlY6VFrbgdLBPWTE5twkkKSSinkii3N8t7MGzvskhC5jrvOQNAi0RFTwbmtJ5V9AizD36MPaFSnZiTfXy9NpUMtqVPC5H7)  
[renewablesnow](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEDELSywgCISEY5TeT340FTHUsOYMxnMC85MUPiFevNRfqG1EEfCK4fbU7HMjPWML-fC5tyKNi07xB-Bm2T4NKwgds5VV_fpFo7bny8jZLINwaL7pDBSjCXziU-g1n5zzAJ2c8CpF-GbXnFPxXff2pbHOK7UYnOakFUQmT5xt8z3VPvIMbXU-e6)  
[rmi](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFy7IqcJFKp-qdHgz66JyOGGKrSXrPqw5Pn-mfj5S-Gh5SUay23J7Y91S7V7AuIm0e-BR9EnIV53DZ_SpYJ9cSZyUK0TEOid8RL3tkfcqlby7JLpr9XjnW6T-mT-N9r-XniYTGXLfrAjy5BEaJLxx3REea7q69fbC5CPtM7XQ96lY00dUZ09DTfagUiiww=)  
[renewablesnow](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGFUd0WQ4r2SNardA4nkx7Ns7WV6m0wy1JD34jIEreXzvgjwwcoGf_qE31uKxBzuMXMuzXDGAbm_q4OqvdheDidzypVPcbeDjK-Fqp_gmq5AEF8CZ3wGnwPh6tNpRZ6IbT_zRdukoR7nKnzQVhZ99q0EBzw9bNzwuxUZJxt4Vb_XokmfhvkqBFT-bmwkEnFODRr0iiu510=)  
[jce](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGaQENDstIPWm7WqrhHftFG6H5b-Y5Egk3Sd9ZtC6-Cp7O8Xt5wzVqYGNyZH8sRhz2Hj2vM_yXuS4nq0OzmxV_wkaqjWPMdLGSqNgzxKW-nMHY5cA331shiMasXZUQrEzkOl6R6Av12RqoA4tx6PYcLg8i01DZ9Mo0UDrQ0E7RK6oqwlWG_wU_dqvB2MP0397isXcGcK3l3Pe2uhtH8dK0wdtFrLVuf0x0tJcU=)  
[edf-re](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGjNSIhzmvKm8-mdlFFX0jWxujYxrglurO8nRDeQt5uDAVCSjlhZJ7Qth5ea4SJfyDwhy_5MM5SHI7qw_xba9Zt4AweS_F_HEAlTxhzswQDgByJgOFth0dEYBa4RHvz0rjCCxbU09tTF5p-zGWblb3QEnqRjGgW2i18YfYyqZ7t4I_8Sf9Kw1ABaUyMv8k3bk0=)  
[windtech-international](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGUvjHwuxGCvQ7V1RwtD--xg0xsqYPjpJq9_u5OBxSSo_KyQKuidIiqnk4cb1MVsz-g0uiWRw7mZGlvowz_fTy3vZMt8TURV-70e1U29io6XN0abb8qmLLLD-RpSQ6QZEYsDRf-nnzN3qZnwQkDjbf2dd624Ad9Zo8f0Z7Blv56AUjxdwT5gCnOljAuTw9uu_k0He0nJ0sqnA4Wvnqr0yPDwXerdm6f32lewWTC)  
[nawindpower](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEKw9UMBWyRAAiyr9KXXLrrzVODAgxSAQme3q0Kqsy-RG8lIF0zb5DWEkWDbtZIa0nqonOilmZjTtVS9iygeKCskl7_VYR5hr-MnDOLgzdkUGkLd4espPTVa8GabtxWG-KzIw-dE0Cm8Ao66R9J2gry_x3CCUX8ncLLw2S7ivdvqnCU8fmfCg==)  
[esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF76A0Q_3YCt36T9jOTYjafSgIy3xHfQmxJn6ZQBJ7RSqbmqbk58xI32-jYDdUMwAdwLtE_541NQuhApHMxTh_m-OjBR5046h2Syhx5_D-Xqo5u9E1gHBjWIxokgJM1F3QbY4HD4G5pgg7lOMRQi3jRayhywbvWAf-4AmW5WhJ-qA9hxwkVHsNwo1iOf4D_i99ccKjF)  
[wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFOLw3KkVQL00jLIvpWvQqfXGGSTFQxka7HaG-nKWuZj64ZozMPARj5p1lgn-grH_IwoViyBkYK--OEkLmjhKoQ8-Sx7fniSQLqitdBEzqYNN9vumJktl53gHflYa_FPq1PdpJ4by09mcU3UGjT78RDS-MOlywIrtXKAv_K_J4qCC7sM1cUnusnetsU6PZa3TgyxGDpjd2DdFs3laPxfFp5psbA0yxwKlo3Jh9_3p8=)  
[homedepot](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFloWJ7nrgrmVIF3yUbhwCjcinmbJXJwGN_ZpqAETsXrz2Teu6j5dErpUnmrhs2TTrU3JjSi2mpd3se_uSJaGM8OWRUd6R4s6cejxIBdzDZpqiod5f-nDZMpP0f6BxeIvLxdMFRTlKoqrvlYr6uy4O05b0qTFz0TbCK9pQDoqx5jkeCEQ==)  
[energyindustryreview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG4JdH5UCbyM-oFJ5W3aRhI-IINM6S1vaE2zR2on6llAPVx4S5FgHJuHVGFYPGvSb7Tl4Vlh0aD9sw55hOI3Ve0RJOQlQMAEdsr_l_Stk-oUCgxQUvSWNCWmirdGkyj9Lq5SAIfdGLGmgw4iIzfvupJHpdobMpsAXKmzAPiVvHw8YSAyxDVET3dfi0DFOHbGbZROkTdydZIoQ34-d_0L1fyXSyYuKRGd3mFwT305HxRr7kv)  
[packagingguruji](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEF5dJ45hUZzoFUfTkvlNTV4zj6iG_9vWTXEdsqIgpCWNp7eSKqBx0URSg1xXe2MREKXZFp2da3JDM6VhCl2nY4Jab7sbJGZ-FrPOl3S-du1rs9aLb_ghh2Ft-9yV-44atgkVhAroBbrrhyg4IEOJM1NHdSJbvObA==)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEQ31OPsEuj1Fh4kuhwW3kCMmuBvvW7vcyBDLVxcHdrfFQXGf5-h81_UWnd1vC7x9tWKT6vDqRFs89EKehyveJFH_L7vQgWzFYuLTIoRL4cwJ4SaDKyjkHgWAwLGLjusuSp5gXRmQ==)  
[knowesg](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFQWTX4wIzjsGW4vUyZq0gEnTlcGWVtKWDK7D0yiqnVNJTC9w2JJyb4kuVDsa0bhJk8wwDfcsC73K_UKXfCvI0FMGpMzDJrc-Lb5iwHHA8UEbCN8tvor3mVZL2EHQuvuRDdIMGh5vW2_o9WvfLIVBR1qde3WkF5HayNHgCb_35s1OlhXm7gy-rTYhivI-uW5-jyQQ5tBezvtxWQVw==)  
[ewg](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFfVD1sZJTp48WCVYtWlE_0sGD1ck083RVtAlFk7w09K9zmP7xOHtrbmzRRpYVBn7iP_yd5Ma3LYT5YbyJPTqXjL_ke_SPOmIn16E_lcv6BJbJOpw7y5apEKuhD2FtLaTTIF_nK2vrcbJLbU1CDmCLG06iWDNEg5cDAQ6LQ-0-4odz1bZniqnR9f5p2dlbMM5SqAoWNZ91XtoawG5OIbk7RmGTaMJ-x89k866hi)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEmqwNspLMFXTgU-m_rICN0pifJbqGaebR2x_qkuytB88FLAB3v0isnXkMPNSd5x2i_NeNxBnElSMxsKiG7tzrP597TzQFVGhfcKMXcnN-oOJwRDVVilS0nTNW5YFUnOC6RsaQxYVT0mI3uj69Aye1h_tNDhFUAlfWKl8FZAsJFPj6eqL8hE24=)  
[carbon-pulse](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFipnZrg81cggN-RC9eOjMFofcDgKwhDhiq3CdalLp10oZbfLCnuGTziuF8m3fKpQvD_35rdqxghJOvTvJUQlSxskbGUidy4I91hYqDatH_vzEF5Sx_wDhvC2s=)  
[carbonherald](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGnr3jNv1PyxaFz_EvHooQ3VRJGcii4gvha3619_qkpvEg9PWUGdIwtz2acIZZs8P3zcAarXT8pNeeMnE0q_H8Guox3ZoC7KAyCRJ4MvdE1_4ilWiMNLDOzIgIVsYVxw_sP9rSQ9pbBBFyqvdJKIdCCxBaQxnz4iN-rzQ==)  
[greencentury](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFCZSS9tn4MWPJ4HnHhEx9OrIvJ-IAlKzpZoVPy0Zboaicrw4294GxT3EPueAL5CpGnyYmmat9T6N-k1dA6DY5TxMRJi3-zesKSYQXVls2iD9V1dmW3Y7y1MY7mzs2M__ydFQBOvv8iREPwbnZcQ2lVpQfBxmrr4JBeWxq4W6rpqh7s2t6nLC6kfEFO)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFnUGmqdTJbAk0N6ZSZ-AazHc9BcAIe0KgHQwEA5Q_4bFoAy6O4KfqQvMl3uBAQ6aG8YfRHIWYw4jdMtF9FUtgLbYrDIABWDjuaq4KGQT-dxycBRgnVE-EVCpchKnWB38bODb1jDKmtKf8=)  
[macrotrends](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHpCrO-9fCHzn3_IP2r14XP1bD-IzFAXnj5N6O7Ff14F4qSznVp29a6SSd7ZIVxyHjQMGcASDfpYVABq1JoG6gY5FjELqy373PtgvLQROB3kNxL9q3wYs8oYXQPKI6tV1t065HVuHpWxZIRg-Ca9CmezjSH8fERZyMcZEoH6wJLkRInpEdd)  
[finbox](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQErwdC7-nrsStg4NroyWgMsAmdwfWdBmgpaQGtCCQarnBuSSVHmb5_m3RJZJQ4kLGzaYAOffxBZZ_tQ9P03r8Yvxhc5MRK1ti2tq91P6G3MEuNJMXl1J6Hjp88eacq-Xs2tovnCzgRL)  
[circularcollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzT9mdzm8yoYZc4ExpF9AQ6FtK4GdKbV_RB0xYEnaepct15N6fo1rd_oMqTmsfEPVBg5dbPxex25KiOCFt9sVrR8XxlFmqGVxzAj42e3tXzY78nHtDj6-nZEkXeJzA2qgaELAqIS2zHAAPTJAepkyv4ADPm_cji-CvjZbUmZxvQTogrsk8XwyLIz32_mxqfg==)  
[esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHNEbPLZp0dsc_ti4E5gw-7BaMsoi4DFKktRqRKrbuiaoBm2yH6UGZww-vwK98J7igdKdK67jupfTPjwTYmrfUjqdxXWamt9Ewt4CbMnImLo5cFTShpoNJwbbnCsipey0WdC2Pu6oAlvfDbrbFx-BiAbt1StsWKLxw7NRWYiO-IGRfCqSE_T2NssG9P1Di50Oe0hWEuoA==)  
[wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGMb29CQaanGVrVNpymob0r56eT4b8UaLFt8sdtwBrK5tRcFr2LW1zOXxpAuviJFVQ7SYvFMYZOkP4NZ54gyNGob2RQTBTJvjez5bHfNOduPuuZx4vuR6EuaaoPcoAdoU_2yp89MI67xd1Wt0JDD6eSORYYEehxrQnmUGQVZ6SfkkFaVxZgMXhwG9eMT-PVZ5Vip9kKKq_BnWuFY7oroT39_ZLofYLq9fTVVQYH0RVT)  
[esgnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFKyTfqJPvycUuU3D6D_u453EgkdA638UYUOfm_sNRTO9h5UXFkrx4rIjpe3ShLIyg4rs6S0nz6oraacH3Fv9mEXDJigJKZx3pMygJWexl9u7DLac3J9-usot7J8BMYjIdI8B4aJvvsG0KdgTA9moLaKTQZXAJox_MNYAT8mtNRRot4CmKOJ15doS8vaQjCfZcjwLSyJOI=)  
[dcfmodeling](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEjaoKzTUFFLKIH4TI2V5sbboay3rLx0nuDvBIU3sIO1j0j3D7Pj5ek8JN1zFmWtqrYeqejDYFqC8FaU_u3JTsOGUXQgjocJakvhzygfmTqwZopjmgRhQNMHsqmo5SVBSREMzlz7TXHUIuJW17WxTs=)  
[responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHEosLDo-yYo6v9XB2uGTkQvmVimWuQlXbrqBJhKmm-jXKB-Rrz8chocIQ6wt8pkopZwPfHnUJmVg-QmvQizw1GdzoSW9GwExcflsKYW8nQ7FSLe5tuxPZAlhMZ1oD9KwCzn_ZR_YL4LYft5usTMsPN31Oktvl17jr6VdB62Rfpq7BNoeBJot7gdNdgINX-1fGV7iMTa_DzuFRrjg==)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHVnkN_GQMmwDwJSB0hxCiWZG877ikgvSEc-sTn55-xKDnw_dwyMwhF8JDT4ZMC50h-B-7hZC0KzOE11DlIW91NP2Mp1fFQ5GuheZObu81LtJ0N--tXajxG-1siqWZ6JSghrliJJafaBRYiyGM7MmorEcMiQTm0MhlRx95T3H1-GR3HZFs0-aPR)  
[sprichards](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGQBwYLmpuxABUMLExtE6OAkCtIY-h7KrHlNzdVE_L4avJ6EGQd9nXKjqxxl9as0JrAw5OaHMTovJvKez5Etwr9-23lbogy5JvdJn_Q3WThE19w_np74cwEc_UxLDvhzdwMuHXjkOZUqCQ0G_EyxcaMBCr6Hj4fMft9HcvCB4KqtShOXBMpjBwVfOgxCEvt4wDnnU4MvCS_uHuP0hJIzZ4u)  
[svanteinc](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFNex9LWz7hAoNsCp4baJAK5N8zgQ9ENPWBwi6qnbc_TeEjPD-Gbtn7Ovpci74dWcpy6cmYxGxCjDU5jFvP8LE4XToiPYFtwXqnX3F2Ft9G68VVIgExQrZ4rLb-TurTgIBnNDBdR1WaLqMBcXOT6XpV2c2ethF3UGapIRjhXHNHGgs7Hu8QseL5V6n3GV3gy7vyq8sXKc83bZkrUQ==)  
[cemex](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHtCk0C_PKzI0tOiVdljzLCkkaqT8UxocWWuc12mfodDLGdmJt4S9SUwFI4GIWyDbVvwJrsy_OW-jQwCpHzP29KrYw4x9bpi2Wp8sKOqbg5xEOBHcLExr6qhDfI6tDQNW7EUWomqbDva-PFE2DPuh5VcUFzWEGso4A8NCoZQrBxBO8mI9Ql)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHR4NqeTMhABBcRbuaaHRehjCH2rkMVEW8No-mjQY_Yj9LYUaZG99Mr39-ubkm49dHlayZNnUo8gG9FmNZXV33d1dY3CxlEAzn7jf0HO-zAomwrI78ggXmDF3Mb_J58NWrJZZMbxE_WAj7gwAESIII0YTGn9CJHetf7_H3eMTs-PZjx0SJ-0rMUl7XN3NwXI5ku7m-3igGUi3YFGvaVSa2uilFVDxs=)  
[scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHCnSiGRYPHDtyUZF8WUyYbqnIqnt-_wFKv_7Vn0hzxOeCWv8ODG7MhNPNYwRbiE9sbDZtg1IfYmp5VulfHsXJTeZ6TbQSupBFqss_-S1Q29t8bPc46g0cfqECNDQxYosdRn3vTsFnIsb4fml0=)  
[techsciresearch](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHDH7u9EBoMUp-zr3o-EGtveKY6oN8UYtwCovWTeC08Nz8j8Y6cGaWjsYC2lOfRz8RBMI2RV1iYidt3GaWaR6Pnvxkl1RkCl6EIFOcC8Y3de0dIaOEeiI481hGoYlyMqUSYgU7sthJIhbZ6MkgGu8za3XxkT05NkF-lb8tZClY1rx-Sgy2fMKoC3TfjHubwl8Iyuq30t7_lpkZzCGHTmD6pyix6NMlv2-DgXtWY95LZTAeq6zGvZ6IUGrUKO3DCeA==)  
[csrwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF9-ms9tv1Us4-Apzu-xg36Wx0V1XvNKIJMSYJe1YXUHs-aC8LbXir0gVewoopj3-c0xCPoe8X2_cgWN-WSFse9WJNdILP0-Gt4pPV4GF4zXKMULbHbqC_RWHnqyds6BRG4J60EhJKoOP_4EaqKAmRQvH6l_3uHDxqJWDAZmrol6EF6bACt5O65yMoJYNjTMWTNYMVgOVM5U761)  
[cloudfront](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGIlDalpWz17n3umhQJOW5SFhY4iZAel4U0ZzZ2IB3t9JUdn9hgEhxQRZqN695gEbTWl_pCheG4Xdx97wcaArq270X7qMEOsoDTu9DyFrrZJVNaGgZJn0xXRSZg0bYLFFN3uuq51edfr32onycUhR9CC9e41xwNiOplmaevJk93vpAvuVxNh17JBbLCjQI1aEBoDAnUUJzCkt1YE0mNBnS175-U-wbMvjORNuTo0n8bkn6l28_eJmlcp2aocPLHfuL4roYrj7HJ24CG6Fgw)  
[prnewswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGYe4ANLkthNvNeTcTSVPMh2mDlSjl0gGVbn9AHzdqmfgJXNABv_sOEaM8TH6GU4kz0KOitBiTZhibYPzP4xslPFBm7z5NQZmk9E8lE_u7vPmzX6SumTOV10O3BAedaxRM9liWGnrM7a6QoLBCGaa5BWCWnRTkCNh7XDLvGiC_qSNb7pbol1Htqp-GVzNk_6JA75c9oK6IDftqWAOsWToBgMIlw)  
[sustainableplastics](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGQ832Nxuy_cc0YK_h-ZuoSJDahpcVUEmFzi50siSsyWUeTsO9MWJoF74kDwo2y2PIhQ49oYzbWgEpEl5j3tq7TgnS9xlF_QIQAXJLamGMDTmdq18BoIFk-o4DIcMPMIhWa3ikasJANql62q5UrHejfALu7EN_Y9rDTSAsgj37-8dt1HxP0f-Q=)  
[proxyvote](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE0fEHFOuF0FsjTMt-t0pkSh-LS1o5jgpVpDMCXHmlaUaek-cKrTqZ4LEc68BmEhUdZ7qNkPJL8zb9SmBRG-mvnWTIFNPG0eQWc5S1gyJfUlQDcZTs0ovv_kCieEUpK5XXqKTmPRCofldzICBfEV1pPGnkv79LFACxhSh3T5M5MWvbL61gALy0L)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHIKevCFAl69SkZaQX1hxt4x1FOv_CGVQS5qIEwzsXAFHIbq3vCgxDrDv-WknLO4Sl--1mkaEJW-4AMMsgsg_VNcwOM2KGzAqktx63GA1CkLs1t587M_BylxbAL9SQBQlqMDyoycoidydosdlIai-tAw4Js0QU1rLNRr5rv-RekGGI6zH34QB4cwgboMco5MNDAnwz-tg==)  
[esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQECUz1INLoQx4bqsy0CuymIbDBys9fVjlSf1p3-9rFUB9klUjREodifGKZ58zMW7tHiSfiWeim2uZAQxI_9arxBVATPsHqNqyX7acU_FRBfeSd-995zZRTo3jNmetfX5aPusAtNfpIFW5rsoSwBq56uACm-zc9pqcNcbqUOaRRbbCSfsccgp0qUhYxFPq19rPkqtWckmw==)  
[wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLOa6zCaiIcCrlH7SmogmVdF3RxQ9HDhjQJFAAu1m8ZBS6JcXTVxtD_FuAw7_G5qvAmGFVPBBbUxcZvbmilPfhsoBvwJZDaEmuwz6fjaSplFUIRMB3VkgL7zHbmcNjfU_cBEn3W2rgDBNpohHayqN4HlTJiMGmboaIJ4ddcUD9w9ACy5GJT_e_FWP_doyvY2hpu2iV60j655gbnVJHkj-GMFaLgaRQrfv2_-ZyOvjI)  
[3m](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpeq3TiNzhZ7rsXhJcd92lhE0jDHpZ7k18pruwRHchQ6UShj7PAGvDMnHKBmq_eJY_ZBNPNfEeRGNDuoVA-HVsVR7cZaEa1LvSgTdRKQgfXI7tiDczH92PVctjUkyL4cVqRthoJdJ4IJE=)  
[net0tracker](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF2sR1kzZz_3zvlz6vAmCFJNlPaXgJ0n9uCkx0ngW8iOz1BrK9GsGiaT8bOPwl1RFNBDufLP7t_tyorGrvFveHv-8yoBzk-MxWLJVXBQmveyUaPQe_g0dHhoSnx-RJCQvW2n8kZnbE=)  
[smartenergydecisions](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGPJgRqjWEXigkzD119q5mY6V8hLEa_jWaUes7OoMtPH0t_MEQiIxb-S5gwbtv_MXsKacGDkhoZYOTG5Fl8b_JHDvwzqVYMQbsdFtCqsi01HJH_Rm4D6-0TKwdaCrvvG3Lsteqda3WtPGOd07YXyFowPqZxVwRucxesznMDTNDTOgw=)  
[csrwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEHysRrnaLYGvoYqiajBMg587IQGHn9S18vhEUBoy9Jpv7-1bUwTnyPvvJETo--Ep-vNfon9PnbiVjdcFMjqCfFRoIIOh_-uoIu4TQFLGJVX9KXecrk3jLk-4eHOgj6NZWGZYSRR-KHvycDgm4-D_FKutTQyd2HL2PwpH5jFCtfkniBnJlGBJ7tGyyhx1W0hsSLzPcSnWG6hoZ5)  
[sustainable-energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHTysOYU_rYxeR7W67mwa-q4-anGIMNkZ3nphLqyN9ygiLBg0SQld3j2kqKxWFdF10aXd_weDoP6Nm-O_JYVKnusyX4h56ESJLbSSt07Dclzw5Tibd_hkNXc9xpDHQiS5GvqjTH4CoT8KXljJkqx05ltX6keYCiV-uN4MJVqac8RSyQ41lUSLUmna1hNXHtWIc-2dRcn1UhdTmnFiI6IkJ4h55Gb7k0Sw==)  
[researchpublish](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF2tEVBrFL3o5FE7xGvSB-XVzkflLM8UJ-kApW0MmL__AM05Zth_nkCL3UjK-wbQvXDfqVnSePMs49EdGjy-UnCAtMV1XpIPP940Og67Jb81ZE7spil1NCtm4xocL_2S9cLanJhGOoIdipgBNIu_No_0XRoZGNRVPO0NF53DjDql6zRhtxVpl1ZIfHbbWo7so_lLX-piWS8UogLJtrqpQ==)  
[cleanenergyministerial](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEzUrS2Mgt1C0tPmdgfoxYiGeK9s0tL0kXE6eX76aM8z0YO6a7OftuD7377g5IB2Wn5PuEMTgadjJgCtc6MY1T_KzG_PMZwLR8730niYEMXx63Xm8BUK211elJGdXyCcd9plJD2i9XcVKeSZ__R7VwBa7KF3UE0sorvn_IGqREhx-UBXGwwHCBw2ea_GXooHnEVuQS561hD6Q==)  
[emerson](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEJpjSeud4871cSVkHEf__HACEzMvH5kGDQiApMvJKceHxK5KZJFCbsmD0wjNz1ScORec-LcQCOxvUDUD22sgjj83SUS9J3GrpkPCk1uKSMfunYCLxwA_dUlJP8-smiMCuT2NXE1tvdN289ZJkVYVQXaorWis2Vueo8vlwSUhCa8oRSxUD1BKRDg3zVrqtuwle7wWpOSngXFgbxIb38mO1L6qzXDZk6Ur6eYdX9sjX1An3H1mZAqcJj6ZUM9w==)  
[investigatemidwest](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG5mxUGuk4JAG7WmrM_7q3ZkGub4OUxHK4d2q0R08m9P4I-_twtloX7VLKg1VCRrGau6hQGzjFmOoeWMDzkg-o4MeFK3--YDVxlUzf5W1W2_xkkmRcOTno93lr124j75t0uN6m0sl5X2ItrauN0bBjdTUBTywYnjicNykZBgyOr--HVEfG6pJeRuEAdFODsvfzYWSMhvPZe0zdXRrXHZb_D)  
[net0tracker](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEc02aFp1GTdmOC8_jNbFesTObMjCVt1cRnghMo6WWSppnL8GbVHQwUq8VFAe-f-aFd3EGmVpuzTgtebh_gT6jbL2azrHx1yJqhJgXkxratKbfDka3STiOPCECHnGqWetctRvveDVd14lFOcKEYww==)  
[inkworldmagazine](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGqRb2doorpB7558y1MYsoikuInmO2JIcinkAyDtsYjLlW8qixl-ZcqfSGRyqok9Nnb2p1ZTFfE_x4cD88bbpXpzB8Ff74ERyluqbLsI2NHpkNJ9FEEFV1H2o-5zkPfU0zm9ssZ0lFStvU81s7DHF6aipFHSQRpl0zGyiZzRkctydKIaof0XOlJtqpQgFTBQIc8dZBqGUirPo4NFlqJlXd3D3UVbSN34bRHj-n4BW-0QHEE7bI=)  
[esgnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHLlcx0zYUj92qINJCwksBK7-J03BpxNwgE_WDIoaCRA91ShZ_YrINH0lW_6YDP8J1gCzBKWzq3YZxQLj0MRLGlgebRqZlbAOSeCevHHtAj7A5QiddiswutLqOB8EuW-hFiN59o8W13pzZZQ0AJuEEEEwqzjy-BQmFnDIY=)  
[smartenergydecisions](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH9BgKI-w4VWucd_VoAkivymnx0MBx36qa3x1mlctdCds9s5d0ZN5_lfa_i9Fq31t5PfH6P52nkdbHdmt9JYaMMnT93GkgwMFm6GUeH8KyqxNGCWcsMJdNnSHmN7kquX4_WdqpJ27q2IL5Eiww_dPw9HhtHWf2cg2HwiDR1xp5Wn80r7iWO3zUcAOoKMwHgOw==)  
[dupont](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFVa7x3RgcbgnK67xcoWVDBKe_82dXbaZim8fjzo1yeO8e5LdOGF4CYz9G-Mw3zqzcV-OzgCph5uftAViOuwC6ps1OxNl1roNhb8odzSCpKaJjVSMHyIKM2XWNqXVG5MZyG1psiNAYPNK_zxADszBtYIfGoLT35O6YbhqQ8MayWqtbbFKz585U5uSs1b-qzf08wCbnFiID5Hz2TMkOA1SHh41oq)  
[csrwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEvHd9feXUQW19oBaCqvWrvYeSoCj5NVG9bfAIvtFbViliEIy5sbomrZSAmvvIWtdeWAn5oNkKTRAdRnWTxdC2UP9OgCXTq3QQctJLxJFamFeDDY-C6tFWHXPsIKzqYyLtRQo9O50jephoGeMWCZyZnKMHbTWsK_ggH4T0o9jvZ398z-ThWr4xcV6yc033Wiw46OMav)  
[dupont](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF5MQYcmTjBod7Na7ANsjWmQkqvyZKnA9lU_8TTu7a9FRGSN-SZ0RnQ00wgXUVFZ5XJrlUzmX5nG_4J0-ddTb-IFxq5DvZj0klxf0RInoM7BtjegGX4eOrd8d0oxLZboiviasz3tZ-OihJWLIpJa0QScMDXqbFzt0SaTvIGQVVh0q3VrI-5Kw==)  
[indianchemicalnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFV2APxEVFHxDrc6czQy-q-G7H0MpClTdX5RsH4nKjwT3Nhl9bEwE1YyX5CDKfRNbDXhgxnX7YpmW0gWDf6NCE9SfmSjQZnTxqAMUVRYZNoDiqtKzqcHTiNqEs3YhR5Sj93huTWfTsVjq3nXX_4Bg4YOPP3ltUB3eZxTC8pd9-XonUqtPTbZVa_xxgvKVuuNAFmFm15DqiZ)  
[dupont](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGAokViDDrhZGlwSYSFVxwzz-srxA2WUNYeV3AeW-MHSPJQM6ecK8P_bg9CP9W8kqnezjM6ODDavmSDkus_nNPnDQUeOMRTop-hMG_bnWzb-wmqFhhSsvDtxsL-CL-hlO8ieqDALh4KOC43lESCmwrtPFV5FgtnPFRQdhfPEA87DH-B67e-kJXw0nQjpiUJMeMQpv5H4pnY7lUV-cR6_gT5RvRnz-7YQ9PqAAK8lteU85kjZF-Fkjc1ELUUxEg-iANZG2F25TuJP_I=)  
[financialnigeria](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcka_6mdPBZCxaJG1z7UWKUoX1ILdXdCsibuOUOQUkDQmuqqdxm_tAQ66UxpaTGmOuIKHhLHuO9iwHq5uUVazCY1O1AJQwhrbwerD6sBdz6zKwKmKOHjYmmb4-HgEATcmiUChH5TTYrXghrVJnvu8FcYOnmqdt_H5dyD6CpHsymuGZC_azkE4nv2Yn2oW2ZSqcjZ5m4zs=)  
[net0tracker](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE3N2iQxlFcdsSp2a6dwOLEgqRUyi7jo9YN3O6579nPI-FbJ0po5mAGMV9TQBOc87rTzUoYm7WnOfvdq35sjiq_QMepamQ9Rwm00-tG0hViJuL-24Ix3QqVgviWuv4vpXlI-tuzgW9zrElF-GGIm7nkmcc73sC9)  
[knowesg](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHx4xBZ9yJZLtMupoSI4SoRn7HkTGHNXuJLzEJ46VIxNVQY72jvtoM02DUKd7ZGWexMAdqHO0HOTD8r4nLp9IbwDV2j9l3haaIby2orDYeV-uVuq8vT-4PQA4Y43LyUxV5SQEcnr9CLyGmUayOH3AClJx4euNpkUAoVV7JrtAaG1kq0cQQEpVxqEMN5MUbul0n3bGuNs3RGCGHMWowYQnCqtBtwjt7-7SV1rQVxYd2cFMQ=)  
[esgnews](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHnSRYL07bh-gxSSlV7FMLCmIYNvpyJKFIea-KLN1km2eOGhSlgn7-0HsP0dpyXFVz_iwTUUczJCMqxsEBE6Z30rsUcmJ9-i6j8o5pHqAmQ5P2JXavJ_0ODiYwsVcX_ItF6AwWLfo7W4GESAn2Jel6BUQxZ9P1w2Io7z9wvqKpqnMflUoG13oyQFA==)  
[iaasiaonline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE55tL2RZXm1NOcy7DOmOR6jLS0D4eXS5p4vQ4P14gK2WqGjtk0RtCt7-Sg4LYHVP_rejK_CiwjaaCGf8R6ZE3lqJ0ZKz0TG9MaoigVvd8MSfdMRaetPChDpgG8TyICjIDNP1WjZPz6dUsSdDmBM_BrsjLWqTT2td28q7E=)  
[specialchem](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFK2UnTAIF5W0E9H2pqLohC6IBLEKf5aMqh4WO6O5z-rk9UBQTGYD9akv7J1PrmNo7HA5g1O5EJkDDLxeIkXBvDe7KzsU-SbHWdt7nvVFDxQAM6CsVOR2mYNHJjEJAk7nw7TgXv_mk2z__1tyUux_k5-Hi52TpGnN7jJM1jDNG3D5IKbHMYJRwMc5TJFImRV10d7Xm80grbnqHd5cM5BQxEjUJyhbgiSklN)  
[esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG_sHoMzQWCLQYcun5ad8m5dAu3Th2sWwTz_ieCKQe2dOHBzMUQ4R3gOWaf-cKJvv0BSg6KsbQwqscWADGihatgVyO3oycnOTR64KDtZiFukluwX48ofzJVzisMwvOnNtFT11ON1hnPlngVjLVYkBwgDOiTZ6CdBmrmOUO9LLYdKv_7HMLZkA==)  
[honeywell](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGOiHmpx8ZT1AAqc1QfwhVf9zxat0FbJhRo9gl_lf7lMfU_7CYSiXvZaxPZ6TSy829vjXODiRYLCBquegituMAkmroPd7qwzL7_syZYh9iLdihTSzvI69yKAmBP3upvrKFmF-jQ_DkZkkHUgt8CE1KT5b0FB8gfF_zflZpWZrIymgp7257w_4Q2a6Dthh9BXP2DJwXL5h161Hz9c_3pQW9D07Lxa2LaOHgBVkANFwDJiZyg4bhYUYU2x1WW8E3NgoNhCtHXpnJmaK3VhUoAYfnUC_icd9h0iIxHF4oe)  
[honeywell](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEk4N7joQ_yBx_bW3jyCZYFZMphUNPJafmb5m5ng_tZgBCN5Go5BKlbRCddmzmf_Abt1swE6OCVcvB0azDL5nT-fgpfe9jpZxPvy4CzPrwDJSMBY_j7eJsPriKoKsYsZ28AvcFsEXjrKYSHIlJXKIrN1Eu2YljZDwHDK5P7GVOfT83_gqqehoeqqhksmqTHkFrl1ddpVPg=)  
[reccessary](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEKiPyKmJXZPZ8hQkg4OQlw-3VhcT-9vC_Ai2MlMkQRLRHScKv7q08poGKbz_WnofiRY2Kt85LTRKLONWyy_nPGuIfqb0OuQ_P7rJUG05QdHR4wWBA4BKBhc6myhFaU0n3fgSIDHNOlnaajEqRa5EmzWRGHNvIGhQhGQJ9ZnVZavD-ikdUiFIIqkxo97oh_nfdi1g==)  
[emerson](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHz9cYbdooM8J6MuX6XqCIdQHX62jnpURq3IsgHC_xXYp-5qBYdUDmA_nZCviYoyVWR3NmfBdfULLVpdzhCxZB4p8vENf_Sle_ArB3ol2eIQyQ1KJza8oCBE4VE2Pwc_IyVCMdT7G-HqiYbJ1-4a8KQGuFJIJ6v45jzPBLX-lTg7LicoH5BC3L9WVn6mM_2esFOTscBEBmp1qb7RJ1BvxjtSGeagt1mUg==)  
[esgtoday](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHhPg1xMRhltpoPmzEPXZYgusQcJfRFRzOugc9DcJM0SBHlQLROJ-dKHkQmSa90RaHn-jxgEr5Z_ambdggBi59muZmlJZ6vRay9y0u9J6CVovWDoqh4oM_-Ky27hvKRSmJxyq7be03Rxoap3QEUJxRvDRx_U51T_o6bP2onKBkByumE6r6sTQNAVa9URkdGtVLoJdcL)  
[emr-online](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGH4GfRFUm7YOb6Bp-PXCNL55tUcfNYJYAZgegeJo839X4Xgq6ubimG6GPWpBWTqfl7XJHCMl2N2t_9IQO12rATk3oM7PZwuOtZLx5mXVlDR9i8cTNiTYLPPLzjs_JREv1raT325M0B91-NFT2yjzbmKdn5zYCDTVAoATCUZtRLoYy0u128KKPgf83e4N6yrzVl2jm6VHpFVQ==)

If you’d like, I can:
- Produce a one-page slide or infographic summarizing the score and key evidence for executive review.  
- Drill into one technology (e.g., Svante collaboration, green-hydrogen catalysts, or renewable PPAs) to map current commercial readiness, expected GHG removals or energy output, and timeline to scale.

# Test Import Sustainability Agent

A LangGraph-based agent for sustainability research and analysis.

In [3]:
import sys
import os

# Add research_agent to path
sys.path.insert(0, os.path.join(os.getcwd(), 'sustainability_research_agent'))

In [ ]:
from configuration import Configuration
from graph import graph

config = Configuration

state = graph.invoke(
        {"messages": [{"role": "user", "content": formated_prompt}]}
    )
report = state["messages"][-1].content

2025-08-20 11:27:32,826 - research_agent - INFO - 🚀 Starting 🔍 Query Generation
2025-08-20 11:27:32,828 - research_agent - INFO -    State Counts: {'search_queries_count': 0, 'research_results_count': 0, 'sources_count': 0, 'research_loop': 0, 'messages_count': 1}
2025-08-20 11:27:32,830 - research_agent - INFO -    Input Query: Research 3M's adoption of innovative sustainable technologies and provide an overall score (0-100) based on the criteria below:

## RESEARCH AREAS:

1. **Key Technologies Adopted (25 points)**: What specific sustainable technologies (renewable energy, energy efficiency, carbon management, sustainabl...
2025-08-20 11:27:32,832 - research_agent - INFO - 🤖 🔍 Query Generation: Starting LLM call...
2025-08-20 11:27:34,336 - research_agent - INFO - ✅ Completed 🔍 Query Generation in 1.50s
2025-08-20 11:27:34,336 - research_agent - INFO -    Generated Queries: ['3M sustainable technology initiatives 2022-2025', '3M environmental impact metrics 2022-2025', '3M sustainab

In [5]:
from IPython.display import Markdown

Markdown(report)

SUSTAINABLE TECHNOLOGY ADOPTION SCORE: [89]/100

Technology Adoption: [23]/25
Innovation Leadership: [21]/25
Measurable Impact: [23]/25
Future Strategy: [14]/15
Industry Position: [8]/10

Overview
3M demonstrates a broad and ambitious set of sustainable-technology initiatives spanning renewable energy, energy efficiency, water and waste management, sustainable materials, and chemicals strategy. The company reports substantial, verifiable progress across Scope 1-2 and meaningful Scope 3 reductions, rapid advancement of renewable energy in operations, and a structured future roadmap anchored in its “Science for Circular, Climate, and Community” focus and the 3M eXcellence operating model. Notable strengths include a validated near-term carbon target by the Science Based Targets initiative (SBTi), a public commitment to carbon neutrality by 2050, aggressive plastic-reduction and water-efficiency goals, and PFAS remediation efforts. Some benchmarking against peers shows 3M performing strongly in renewables and emissions accountability, though direct, apples-to-apples comparisons depend on methodology. Key 2022–2025 examples and metrics are cited below.

Key Technologies Adopted (25 points → 23)
What technologies has 3M implemented at scale?
- Renewable energy and energy management
  - 3M achieved 50% renewable energy in its operations ahead of schedule and surpassed its 2025 goal; by 2023 it was at 56.2% renewable electricity globally, with 2024 data indicating 62.9% of total electricity use from sites powered by 100% renewable electricity. This reflects broad-scale deployment of renewable power across global sites. [sc ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==), [AccessNewsWire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcJy6Y2HKc1P5kTkpk6IlCQDn0gmret0kYrucpTAX-_ONVPwPgM-QGRjx5HuucksflhMKLF4RSuteUx8CNHP5hw2-L0lcKfuIYdwGk4374O-b8dbBqekjO9aL-x0iHFZ6ZYLGpQ9fY4_qG79zVsLCZQtIOqZzOArwQ13_-Ko2uEX3ss87lRLjXWLEbShCuyIjgTCI7fnkRe7WsVP_yTpd65UywdVUn6ooo7CgJb1R6pOrWA6IY7eLpbKM4XPoIPv3vVkiKuv8-Kg==), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=)
  - Renewable energy targets include 100% renewable electricity by 2050, with substantial progress toward the 2025 milestone. [responsibilityreports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExk5kNwlFlid5OHmfTREiV2jYxpBuwFO19GJMCQnpFv-gq7luNdXSIHL5oJr3MPfzdVzp2v5qt_pX5QhVz8TzsQvCt7XSCmlc5D5g4MmKbXBk-djWWhWFg825RFl9j4znm2T3QLDd7B1zjpzkORZXnSrAOPGLi7A1FOpuJVPhXqYXU3hGeL2ifF56TXYfhMbx9nQe5dMNdIk2h), [CSRwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZOedEeKW13c0i4BPWHm41T8sh06aU9gFW0sgzylfIelLiocle0ARsZUJP3OKEaNCEopE6asH_aupFxcHGAJw60-CJidSN9eBZ38DCQ16LAf-ZkZDdFQPCXBPP1VpGXSq9lIk8BLfGGi1qO8YBfjHqkB5yLNwrUE1sQJLx5rUCzXPbNDvpEiUzv2uJu7mIoxDKFchx2A95Tiuj)
- Water purification and efficiency technologies
  - 3M is installing advanced water purification technologies at its largest water-using locations, with completion/operationalization expected around 2024. Water efficiency improved by 21.4% since 2019 (through 2024 data). [wbcsd](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [startribune](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGFX_0dOi8MovdhE273N73vDjrP5rtcQalt2UchbUpqto-uhhnn44IVGi4OTDHDhNRS31Od7XrH4LlfGUVmoVzFCbXUAaBeAgqGASJlQ5BpJ_c41kjB4zX95NrAYyfv9ZLcgXDzxO72jE2-bLt9cjA_onEwh7ZFeu6Fw0pjZxfO3jDAavAhbdY-4BbmThluBdDKHrbERqKanmMoiJYS3XUUV3JUYLJD9NXFiuiKgHJMgbBMLWvZUC6c8WYSlnPV), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcrdI7iGrt8DXKMgkjR-IYVPb4DQhYk8tFJ6x4qIQg_iwrNECzIPhx19psPiqkDLxs_-0Pg0JuY27Y9FohRZ_NywetvznKmXU8hq9wh_6WP0H7Ld6L7aTEoX3D-sieUfF-FqaSuvrjYabHUTt_6KfL51h3s-e0-E2RqYAcTPT4CFxzZMVhugen)
- PFAS reduction technologies and remediation
  - PFAS (forever chemicals) plan includes stopping PFAS manufacturing and use across products by end of 2025, plus investments in water treatment (Granular Activated Carbon, Reverse Osmosis, Ion Exchange, Thermal Oxidation) to reduce PFAS discharges. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEGnvaPWRix3HotMzOznmOiq2Yxcj3XTOUeDK-scVwV3JY1WqzNhLclIUKhjwTMupgv7zf_WTRV8w5OY3X7HoHvYdc5tkdPl_yLvRBnXm0_5EBsuov6wbbRZyQlLd0tIVd9e516o_HMeyX7MZIlxHi0qWnDS51zB298mkIPsgCC_FdNGpv4i64vAC8=), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)
- Plastic reduction and sustainable materials
  - Virgin fossil-based plastic use reduced by 98.2 million pounds since 2021; targeted reduction of 125 million pounds by 2025. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==)
- Waste reduction and circular economy
  - By end of 2021, 45.2% of sites had zero-landfill status; a 10% additional manufacturing-waste reduction target by 2025 is in place. [CSRwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZOedEeKW13c0i4BPWHm41T8sh06aU9gFW0sgzylfIelLiocle0ARsZUJP3OKEaNCEopE6asH_aupFxcHGAJw60-CJidSN9eBZ38DCQ16LAf-ZkZDdFQPCXBPP1VpGXSq9lIk8BLfGGi1qO8YBfjHqkB5yLNwrUE1sQJLx5rUCzXPbNDvpEiUzv2uJu7mIoxDKFchx2A95Tiuj), [Sprichards](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEYNhkbikxGKhxGNj3_MztbUCnnvdFcWRbDmdWVshiv1nt17om57DSE3JrvnwJqNaDGxT4skyscaj9ML33hhUsRJgDw2aWl31aSHOKrhFNVWBmKARctRgom4JQUNPNu917gGsMnb_adqxUAZQueNmANh14j_a8VYETFSanlig7ieNN2wZN1fpBbOoxjkAWQDGo=)
- Product sustainability requirements
  - Every new product must carry a Sustainability Value Commitment to drive reuse, recycling, and reduced resource use. [EnergyIndustryReview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExKstTFp4hGHn2qwwG9tGkeyOzC4OBv7IJK2SGsajUF75ZPOBCT1YqfQq4TLQ1SGTqqjhm0a2tvWd9NccrgdhXCnh_MTIzihgxcD1tVCJqjJBMazZXEhIuAM4Nlly1XoUSmeOBWHLtetZ92aUNGaZbfMjXrm70e2JWq1Lt67CpZjXOOjyTqwL7txX7a548-90icR3cq60RpTZSsCmE2ww=), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFJc1AURpbdLQ8jfnP182JNegdpR2MoTBsWvqlbOO52ZbMs6mLKEuNhqAW8MURbGkiDoGJQjDQv4jGQLl67Bfw0Siq7cKei6mg2OOnusRIjxcUw7LzOGcyEmN3butf1N2b2aEJdYTNQeov4SQxMIcERoUL6ljEKRQMBw9N2bqtUJTUFBgD04tlF)

Why this merits a high technology-adoption score
- Multiple breakthrough technologies are deployed at scale (renewables, advanced water treatment, PFAS remediation, GAC/RO/ion-exchange/treatment technologies, waste reduction and circular design). These are integrated into operations and supply chains with quantifiable targets. [sc ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)

Innovation Leadership (25 points → 21)
What cutting-edge sustainable-technology initiatives, R&D projects, or partnerships is 3M pursuing?
- Science for Circular, Climate, and Community pillars
  - The three-pillar framework guides strategy toward circularity, decarbonization, and community impact; 3M also joined ED Ellen MacArthur Foundation’s Circular Economy 100 (CE100) and maintains supplier and materials traceability initiatives. [3M Vision2045](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyZXiQbyESiFn7GLg8GVBaJTFzxw8hjthuH9iTJAPqknyHDMTEfFecg71fj9ofp0pJAvnSP31t_nunk-RNJZTP6ywaMVnJm_taS6-QLN8S3npT-WA0KYC3WkegGFVJGVlfP_9cqPe-S0OeoqZeYxWLAW4=), [TriplePundit](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHKnKS9TNCg6g2UPgcYU-wnyw-WWuyelyMrk--cbVHe44EYHTACA99eTUcBl6kkznQg-NnSSp_LAC8_NnFTWrzYUwxco9SqI_pC5aZFY0nFgR0xIhPX6NTM24dHkhAhXBEMPHvZmVxJRRGyH_C_IqzPpgjVM4j04WWt-Jg5B8hYqHf_9vttwJoSTb20pwYZ65ACjiPjJaEwpuvPEVGuYrPLdhvW), [CSRJournal](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGPND-1nn4B_XzGpT7Q3y_SOeENf40X5452rIgPYx858jxIU2box7dZVoiT6OcC_c4hx6przR7tsVI_XKKxp1lW6Ry-NsqnN1ge3xCuYuEyOM0xmpcC1qpul2BP0NQjfPnNJV8MyajP_Ido2mFexCv8OhsHqLgj7A==)
- 3M eXcellence operating model
  - A comprehensive operating model focused on margin expansion, innovation acceleration (doubling new product launches), and supply-chain resilience, embedded across the enterprise to sustain a high-performance, innovation-driven culture. [AInvest](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEAWnkMGJVGW-B3ok728rFpizJnQl2ceDuIBRjnv0YcvOfIiXPZ1rixECfkq5cWvPVIYSH1bnAe8ghtYxV8aQSsEjN6H5e73InQsASju2XZ3531fa8yKrrZsW_AnAHGPgL1A8PHwYgLhDaKVMoWc_OOL3AfX83I9CGtE5USqHj3RoLDOtzc-0FsltcJgq00h0ac3FZDKLspZi7b1oy8uC50T-MJ8aY2i_t6), [Investing](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==)
- R&D intensity and future product pipeline
  - Annual R&D investment around 6% of earnings with plans to invest about $3.5 billion in R&D over 2025–2027, targeting climate tech, industrial automation, next-gen electronics, and sustainable packaging. The aim is to launch 1,000 new products. [TriplePundit](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHKnKS9TNCg6g2UPgcYU-wnyw-WWuyelyMrk--cbVHe44EYHTACA99eTUcBl6kkznQg-NnSSp_LAC8_NnFTWrzYUwxco9SqI_pC5aZFY0nFgR0xIhPX6NTM24dHkhAhXBEMPHvZmVxJRRGyH_C_IqzPpgjVM4j04WWt-Jg5B8hYqHf_9vttwJoSTb20pwYZ65ACjiPjJaEwpuvPEVGuYrPLdhvW), [Investing](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==), [CircularCollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF8un6wDql3WuH_ntQiAIt18jycKwdFU5dSLx4dGb27TrD_DSk4eOs8i7aAQjN_8lBUPRX11v4sE2AleM_dS6SfnoPTAKBNU25FbMEas7ln7CINPSEp1SSweA2dALYiMPyKo6t4E7iGhj4pN8nRWSKH0RkEIw5wYerr_93qr_rKSsxz8Tn1SwV6VE7nlyl2)
- Partnerships and external validation
  - SBTi-validated near-term carbon targets for Scope 1-3; collaborations with The Climate Group, WRI, and WBCSD; RE100 membership for 100% renewable electricity. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [EnergyIndustryReview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExKstTFp4hGHn2qwwG9tGkeyOzC4OBv7IJK2SGsajUF75ZPOBCT1YqfQq4TLQ1SGTqqjhm0a2tvWd9NccrgdhXCnh_MTIzihgxcD1tVCJqjJBMazZXEhIuAM4Nlly1XoUSmeOBWHLtetZ92aUNGaZbfMjXrm70e2JWq1Lt67CpZjXOOjyTqwL7txX7a548-90icR3cq60RpTZSsCmE2ww=)

Why this merits a high innovation-leadership score
- 3M is actively pursuing and reporting ambitious, externally validated targets (SBTi), aggressive R&D investments, and a broad ecosystem of partnerships and circular/climate initiatives that go beyond compliance. It also emphasizes a structured, enterprise-wide operating model to scale innovation. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [EnergyIndustryReview](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQExKstTFp4hGHn2qwwG9tGkeyOzC4OBv7IJK2SGsajUF75ZPOBCT1YqfQq4TLQ1SGTqqjhm0a2tvWd9NccrgdhXCnh_MTIzihgxcD1tVCJqjJBMazZXEhIuAM4Nlly1XoUSmeOBWHLtetZ92aUNGaZbfMjXrm70e2JWq1Lt67CpZjXOOjyTqwL7txX7a548-90icR3cq60RpTZSsCmE2ww=)

Measurable Impact (25 points → 23)
What quantifiable environmental and business impacts have resulted?
- GHG emissions reductions (Scope 1+2 and Scope 3)
  - Scope 1+2 reductions: 59.1% since 2019; 52.6% target for 2030 (SBTi). Scope 3 reductions: 30.7% since 2021 (aligning with SBTi). 2024 data confirms substantial progress toward targets. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQETM6ah9H7vwsMWMRBiPcyCwxBAnFGcWdh7X_wW-MPfPGtgsbgNpl47sEEcECwcBLgn_eiBzW_XiUWp-HiNGnQDnGUWZlgMrIB19_WIDWjYKEWy_A-5KZQOjZ_LJcXs1v7zKIx2nTSf4A==), [ESG Today](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFIBf4B_V9hfyvr1dHg7K7Qymj0XCHr1aPgQUwocOBx2l9z7Ecw_o5IQrRO685iWl3WBs9WL1Vzs_JBgUyzNyr60LLIkcO9aZ06dO-hD_2-fJR3XEnlZ-KAN5dcnhK-8Q_URB26gbDd0s0H0u5ajz1poJ-fzdldj2-ug3WSkXGMR-BjY0tTRKUmA6fjHWZt2smb4n2Yw920h8Wz95tAB1QPVxubWsw=)
- Renewable energy progress
  - Renewables share surpassed 50% ahead of the 2025 target; 56.2% in 2023; 62.9% of total electricity use from 100% renewable electricity sites by 2024. [AccessNewsWire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcJy6Y2HKc1P5kTkpk6IlCQDn0gmret0kYrucpTAX-_ONVPwPgM-QGRjx5HuucksflhMKLF4RSuteUx8CNHP5hw2-L0lcKfuIYdwGk4374O-b8dbBqekjO9aL-x0iHFZ6ZYLGpQ9fY4_qG79zVsLCZQtIOqZzOArwQ13_-Ko2uEX3ss87lRLjXWLEbShCuyIjgTCI7fnkRe7WsVP_yTpd65UywdVUn6ooo7CgJb1R6pOrWA6IY7eLpbKM4XPoIPv3vVkiKuv8-Kg==), [sc ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=)
- Water efficiency and use
  - Water use reduced by 21.4% since 2019 (2024 data), with a goal of 25% by 2030; 2.5 billion gallons per year in water-use savings anticipated. Advanced water-treatment installations at major locations completed by 2024. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [StarTribune](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGFX_0dOi8MovdhE273N73vDjrP5rtcQalt2UchbUpqto-uhhnn44IVGi4OTDHDhNRS31Od7XrH4LlfGUVmoVzFCbXUAaBeAgqGASJlQ5BpJ_c41kjB4zX95NrAYyfv9ZLcgXDzxO72jE2-bLt9cjA_onEwh7ZFeu6Fw0pjZxfO3jDAavAhbdY-4BbmThluBdDKHrbERqKanmMoiJYS3XUUV3JUYLJD9NXFiuiKgHJMgbBMLWvZUC6c8WYSlnPV)
- Plastic reduction
  - Virgin fossil-based plastic use reduced by 98.2 million pounds since 2021; 125 million pounds target by 2025. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==)
- Waste and landfill
  - 45.2% of sites at zero landfill status by end of 2021; ongoing goal for >30% zero-landfill sites by 2025. [CSRwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZOedEeKW13c0i4BPWHm41T8sh06aU9gFW0sgzylfIelLiocle0ARsZUJP3OKEaNCEopE6asH_aupFxcHGAJw60-CJidSN9eBZ38DCQ16LAf-ZkZDdFQPCXBPP1VpGXSq9lIk8BLfGGi1qO8YBfjHqkB5yLNwrUE1sQJLx5rUCzXPbNDvpEiUzv2uJu7mIoxDKFchx2A95Tiuj), [Sprichards](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEYNhkbikxGKhxGNj3_MztbUCnnvdFcWRbDmdWVshiv1nt17om57DSE3JrvnwJqNaDGxT4skyscaj9ML33hhUsRJgDw2aWl31aSHOKrhFNVWBmKARctRgom4JQUNPNu917gGsMnb_adqxUAZQueNmANh14j_a8VYETFSanlig7ieNN2wZN1fpBbOoxjkAWQDGo=)
- PFAS remediation and product safety
  - PFAS exit by 2025, with remediation technologies deployed (GAC, RO, Ion Exchange, Thermal Oxidation) and a $1B+ investment toward environmental goals. PFAS discharges reduced by >85% by end of 2022 vs 2019 baseline. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEGnvaPWRix3HotMzOznmOiq2Yxcj3XTOUeDK-scVwV3JY1WqzNhLclIUKhjwTMupgv7zf_WTRV8w5OY3X7HoHvYdc5tkdPl_yLvRBnXm0_5EBsuov6wbbRZyQlLd0tIVd9e516o_HMeyX7MZIlxHi0qWnDS51zB298mkIPsgCC_FdNGpv4i64vAC8=), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=), [PR Newswire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGBIqlD8f_T4OZdjwcTJsikPtA_D9DEKAMtTZlrhWbGZKnWVmUvmuolfHlSjcKT6YcSCNLv7g4CMXpthtJ3w-xk5BfedlD1EnUxtWgPzZBKTM7okJNzKFLi9Uxu4EVxAhh8Nq_w1dcuS4rUOLkzpddHbLmYHDLcA1VjLzVXp9uvYLYK7DBLEpC7hxbXLfIe5ijiqjoaafSafkZFrCGKXHgl5ikCKdf4GZhbFOdQwTqrZqiEyMVXNUE70Nyelkgnnw7wEaC_0e1BTFPrGbllj8ee8D0=)

Why this merits a high-measurable-impact score
- The portfolio contains multiple large, auditable outcomes (percent reductions in Scope 1+2, significant Scope 3 reductions, renewable-energy share milestones, plastic-reduction milestones, and PFAS remediation investments) that meet the criteria of substantial environmental and business impact. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQETM6ah9H7vwsMWMRBiPcyCwxBAnFGcWdh7X_wW-MPfPGtgsbgNpl47sEEcECwcBLgn_eiBzW_XiUWp-HiNGnQDnGUWZlgMrIB19_WIDWjYKEWy_A-5KZQOjZ_LJcXs1v7zKIx2nTSf4A==), [ESG Today](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFIBf4B_V9hfyvr1dHg7K7Qymj0XCHr1aPgQUwocOBx2l9z7Ecw_o5IQrRO685iWl3WBs9WL1Vzs_JBgUyzNyr60LLIkcO9aZ06dO-hD_2-fJR3XEnlZ-KAN5dcnhK-8Q_URB26gbDd0s0H0u5ajz1poJ-fzdldj2-ug3WSkXGMR-BjY0tTRKUmA6fjHWZt2smb4n2Yw920h8Wz95tAB1QPVxubWsw=), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)

Future Strategy (15 points → 14)
What are 3M's documented commitments and roadmap for future sustainable-tech adoption?
- Carbon and climate roadmap
  - Committed to carbon neutrality by 2050; interim targets: 50% reduction in Scope 1+2 by 2030 and 80% by 2040; SBTi-validated near-term targets for 1-3; a climate-transition plan to be published by end of 2025. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEwAzTMyNgDWWI8U2-ALYs8xFlylSq6QrGjDSJVEJLuerfSB-ITBmfVmAu0cROuj5yavS3HPxP8ibB_UZdpGOREYl0GzcO7Q9BpfQdlKqKmQMvZUvAUw5bWLxaZscXI24UQbleQDLAT5E2OuQ9EcCw6XI9Mt5BihmzbuDe_NPopwjZQy0QbEo9QdYEPsDSCYUke1a0VAfyP17FkuS6_CfRte31GCA==), [EnvironmentEnergyLeader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG9oeyp2qE3ha7qlpDP7F3W_Pn6vp4jdU-SbkrtiLssxkcpda-LTifHj2kdhE6UU9ZbjBgHvj7oMBlLUisXERKs2nr9WzlMEV34vVyrtO7b2EDlFxiJJ8UlIH2O-Vb5R84qYgl5Dym2ytVlFxJKOWYEZ94Kosjyy92bfuXxlZGe8jl2lGxi2rQ2QRGCRzIwB2sHNrVAuUSuP6ZRDXqvi6m6MGfHdI0nZrRwiw==), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)
- Water stewardship and plastics roadmap
  - Targets to reduce water use by 25% by 2030; 20% by 2025 (interim); 125 million pounds cut of virgin plastic by 2025; water purification installations completed at major sites by 2024. [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [ESG Today](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZetnOWmgi0QXkofHAaqJdw1HycRNa2ug7RTysSq_ZLVdNeu1edDrmG9JuZg36OLXOiDTR-Q67v4dkiV1uNGbc_o_ieCZkpfhf1C5c-hyTvNgDUXCO_TGzrh_7jOAh6U5sV6k61FP2jXKsb_e-1A28BrjkIqwb_i3fiErMPgYQh_Ju9Qo_4W5Xtyj_pHwpgKdGKdfU-Q==), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcrdI7iGrt8DXKMgkjR-IYVPb4DQhYk8tFJ6x4qIQg_iwrNECzIPhx19psPiqkDLxs_-0Pg0JuY27Y9FohRZ_NywetvznKmXU8hq9wh_6WP0H7Ld6L7aTEoX3D-sieUfF-FqaSuvrjYabHUTt_6KfL51h3s-e0-E2RqYAcTPT4CFxzZMVhugen)
- Waste, circularity, and product-sustainability commitments
  - Zero-landfill targets by 2025 for a substantial portion of sites; Sustainability Value Commitment for every new product; emphasis on circular design. [CSRwire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFZOedEeKW13c0i4BPWHm41T8sh06aU9gFW0sgzylfIelLiocle0ARsZUJP3OKEaNCEopE6asH_aupFxcHGAJw60-CJidSN9eBZ38DCQ16LAf-ZkZDdFQPCXBPP1VpGXSq9lIk8BLfGGi1qO8YBfjHqkB5yLNwrUE1sQJLx5rUCzXPbNDvpEiUzv2uJu7mIoxDKFchx2A95Tiuj), [Slideshare](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFpSenjrquLPPc1GAqiDU7R1ylFR77uNrQcze_OdhLydKUwbFxsEFKZL1InJH_wbCVQIbeNaZWU6yjb25_kCVyr84BdfsBNgxgOa64H5FpROozNcrydGkmX8h6qR4uJCisc_tNz3ibVotDIkaRIFvFGxczspXZH6jDlTTwT7KsnTN3Z6DrIqZL6aFUoQ8mBzTnsqfbLbxCOlAE=)
- 3M eXcellence
  - A formal operating model with explicit targets (e.g., margin expansion, 90% on-time delivery by 2025, increased rate of new product launches) to sustain the sustainability and innovation program. [AInvest](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEAWnkMGJVGW-B3ok728rFpizJnQl2ceDuIBRjnv0YcvOfIiXPZ1rixECfkq5cWvPVIYSH1bnAe8ghtYxV8aQSsEjN6H5e73InQsASju2XZ3531fa8yKrrZsW_AnAHGPgL1A8PHwYgLhDaKVMoWc_OOL3AfX83I9CGtE5USqHj3RoLDOtzc-0FsltcJgq00h0ac3FZDKLspZi7b1oy8uC50T-MJ8aY2i_t6), [Investing](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==)

Why this merits a high-future-strategy score
- 3M has a comprehensive, targets-based roadmap with explicit near-term and long-term commitments, substantial investments, and a dedicated operating model to execute the strategy. The impending climate transition plan (by 2025) and strong governance on product sustainability further strengthen its strategy. [Science for Climate](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH7YP6zbI5-OPIgGfoG0vlaC3gGsd64lQtsxEyz7iKShYmMe8RDR3G0L4xkQ90gURrr-Lmj_SCZYy9Bx4bMte1zDnN9IowiSxdXp9UULnrXh1IO-ssVXN6u351W_a_fgmEvqXVE6wyFFlne8kyXKblUJk6yYmDrNgkakH1nfqUaOed_L9TPfd2_D06Ih3PFqRJlUf16), [3M eXcellence](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==)

Industry Position (10 points → 8)
How does 3M compare to peers (Dow, DSM, Honeywell, Corning) in sustainable tech leadership?
- Benchmarking context shows 3M with strong renewable-energy progress (ahead of 2025 target; 56.2% in 2023; 62.9% of electricity from 100% renewable sites by 2024) and robust GHG reductions (Scope 1+2 down 59.1% since 2019; Scope 3 down 30.7% since 2021). In comparison, peers show varied progress: Dow with 40% renewable electricity by 2022 (and 2025 goal), 20% freshwater reduction target; Honeywell pursuing carbon neutrality by 2035 with ISO 50001 adoption in early sites; Corning and Owens Corning reporting significant emissions reductions and renewables shares. While each company reports its own metrics, 3M appears to be among the leaders in renewables share, validated targets, and PFAS actions, contributing to a competitive position in the sector. [Dow reporting context: [PackagingDive](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHb_K51S1URb74w6x8ScUPxWrtkkWekpcpGfJzy5rq1-pYfdSCdTCD67Fc7u4Plrrz2gWo8Q11JepsEVDBFIttOav9ozGz7gAspct-ZP-Q_u7P0vLeT3zYgpmIs4MgAYcFi572MksDLB8DLISkzUJeW-NJ9pN-_wJDSZJ6fv-or2H1D43lPSaMh8bbRTHLseD6os8BL-bGJSYcSuoL_0mU=)], [Honeywell Responsibility Reports](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHFVIAIxDRGenIuuXRkIIhu2Qnad7Q3RoptxgHF7IINkef3XweYguFGI5PXZtjeBXvrUyXevzHOSmsg0X2Oc0l1OQZ71E7K0lwnYyfoRcSLOCcn7T3Ynxz2a_v-dv6XBQ0Flz1on4D1DWhGyw1LU90gfe-qS6btghp6_Gqi9yrc7md6g-8eoUU9pSkIwq9m8IqKrK6d7Nyd4LnU), [Corning 2028 Scope 1+2 targets](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGM3EDgyX_tAi1TRoXWbjaws4xjuefrPqUJy4io832SS73LCWXoVukASCwz4Quf53xHw_vpT4QeLoGO7xL4AeaSOHhNkeoJ3y3GaLDxDZHqdEi6l8oaXLiegKmFgwFDN-zgK3-gqj_sn_yJ7KbPkJy2_JDP-nC9ainP3SIQmDcjT5A0NXkUYDgXsErymVuhAIRzLrA7VTtJyQw=), [Owens Corning data](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHt9Gt-JLIWyq9v6QPgoBc0ZVapOcCUpwqmIclKo2NKKvCxIkQET_PKFt2gTIG9lbiT7b7t36xcVXAefynjgwvYTBuEZP7mbof42bG_OseUp_Pr86lo1pCiby9Syaz15USIWfLyStLLdQbgP5Mv-jSYAjfNP05i6JuK_IqkwqWi2ZmEU2CGZ_Km7j8EbfdNXs3QPh4rYkvGK6vesIMcDL88_km5Wnron8HqyzK4ByAe4gCk789z7P3nPmmFLXX2KP8=)
- Note on comparability: Companies use different baselines, scopes (1/2/3), and reporting cycles; direct numeric comparability should be interpreted with care. [CDP context](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHCbWSmBJq3wh3h69R-uh8HOkJV95rBFLx686B5LFrJLqwuMWSOaPZvItsfH4sbCxxqbh5HsxV52gufx7fkwOCqXHliphwj4AYw6Yn4y1uColPimZ5lwEgrGXRFX38=)

Why this merits a high industry-position score
- 3M shows industry-leading momentum in renewable energy adoption, validated science-based targets, and aggressive climate and water goals, placing it among the top performers in sustainability leadership, even as peers also push strong agendas. [Science Based Targets initiative (SBTi) validation: [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=)], [RE100 membership: [Sustainable Energy](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEh_WmbWIZw8_8k37qPoJ7lB0XPzYPJR3hruaR2bf6TUmbdouA3TCbBBmBGNY0nzxRUNU-2aqsQtzDYWqLKO4yDiWG0dB4jY30W2YECiP3TcNa5RP3Fd86aL4V8nqbYjkZ9A20oU25fXFb7qWnKL2jwrOBL_pebzorOCPrYy7JM2DBfPGzPPa59wXpQ49xtDesFm8hty_AuPZwORCemTeLrakRiY-xUbg==)]

Final assessment
3M demonstrates a robust and forward-leaning sustainable-technology program, combining scalable technologies (renewables, water purification, PFAS remediation, waste circularity), strong leadership through external validation (SBTi) and partnerships, measurable environmental impacts (significant GHG reductions, water-efficiency gains, and plastic-reduction milestones), a comprehensive future roadmap (carbon neutrality by 2050, ambitious 2030/2040 interim targets, and a climate-transition plan due by 2025), and a competitive industry position relative to peers. While there are ongoing investigations into facility-by-facility metrics and more granular data, the evidence supports a high overall leadership profile in sustainable technology adoption.

Cited sources and recent data (highlights)
- Renewable energy progress and targets: 56.2% in 2023; 62.9% of total electricity from 100% renewable sites by 2024; 50% by 2025 (surpassed). [AccessNewsWire](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcJy6Y2HKc1P5kTkpk6IlCQDn0gmret0kYrucpTAX-_ONVPwPgM-QGRjx5HuucksflhMKLF4RSuteUx8CNHP5hw2-L0lcKfuIYdwGk4374O-b8dbBqekjO9aL-x0iHFZ6ZYLGpQ9fY4_qG79zVsLCZQtIOqZzOArwQ13_-Ko2uEX3ss87lRLjXWLEbShCuyIjgTCI7fnkRe7WsVP_yTpd65UywdVUn6ooo7CgJb1R6pOrWA6IY7eLpbKM4XPoIPv3vVkiKuv8-Kg==), [scottsafetyuk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFovV-TveXmACLw_cSoUiUvLxgKO3x7BUyMqleEAHdi1jnFn-q1sTx7UQsb42Xz4IFYP0PoNvn6QClIxyFRXQyumX7T-UXEQywYnX7ux6Grj1zl3iGN8U4ebYyd9qCqyDprYt9uzUS_ABq7Mf5kdj-d32Yz41tUiVacMLlQScGy6fgqbN2tNFs=)
- GHG reductions and SBTi validation: 59.1% reduction in Scope 1+2 since 2019; 30.7% reduction in Scope 3 since 2021; SBTi validated near-term targets. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQETM6ah9H7vwsMWMRBiPcyCwxBAnFGcWdh7X_wW-MPfPGtgsbgNpl47sEEcECwcBLgn_eiBzW_XiUWp-HiNGnQDnGUWZlgMrIB19_WIDWjYKEWy_A-5KZQOjZ_LJcXs1v7zKIx2nTSf4A==), [EnvironmentEnergyLeader](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEtv4eWupqd3PFqgywBcGVhJRBaIuB9Xku1hgZ9Po4ozB1KsHB_NAtLuDOKnGUwhiheSDtuPVWZM5Cjd79Xc5EMrNmWiqS0HRC3ApeI2xpuDaitakMA4ijLqxDL8yFCdCA6Fe3faF9s8anEyY9D5P6ZySKjkBawHtY-Mb1aLucQXOXqP5LZquF81R8ddNM1Q7WcaZSslqyEuoMBvf84bx_wWerAaaQ267Rf), [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHjd-x7HYLWH0_wobaNP5KRTpqOEg0JuVqCxOvOSZWtlQy5rImyfQfTT-MD748R0r-mIz92RAyG2UKKbKvACm62GilwvHAt8Gx210e4fDVbgVo3LhLufGSHQgPr9xzH2_5c_ZsVacIVDUR2yuhYS750JfnPVkNeB4oFhaqcas-aw01pFQczYuJ9mnyE3uLkUdowWx0ZTiudrPT7XVKdiWKxPJn_wxA=)
- Water management and efficiency: 21.4% reduction in water use since 2019; 25% water-use reduction by 2030 goal; water-purification investments at major locations. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [WB CSD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGpGk_iu2QyYIBzD0l1Yn1-21TihFLeE1OEJ8CWYUemRlZgcvwZA_qeJIPLaWLkhRNe3GiOTNP4p1i6fVg5-oypgF68VHRfB7XS8R1EP4xoUHU_4rQUbZkHb0zEDuVN083jRBDWCf8rhf03x7dxz4_fEuFvZX_hdKU-US-NwzPSIdLqNksdTnk5gQ4a6VxW2HhXWUkwabbDWd5ZuUzTq8TDEs0Wj_l6WHCIorEtjEd), [StarTribune](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGFX_0dOi8MovdhE273N73vDjrP5rtcQalt2UchbUpqto-uhhnn44IVGi4OTDHDhNRS31Od7XrH4LlfGUVmoVzFCbXUAaBeAgqGASJlQ5BpJ_c41kjB4zX95NrAYyfv9ZLcgXDzxO72jE2-bLt9cjA_onEwh7ZFeu6Fw0pjZxfO3jDAavAhbdY-4BbmThluBdDKHrbERqKanmMoiJYS3XUUV3JUYLJD9NXFiuiKgHJMgbBMLWvZUC6c8WYSlnPV)
- Plastic reduction progress: 98.2 million pounds reduction since 2021; 125 million pounds target by 2025. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEiwSThu_G9beuPgfHJ47mYcyRy6YrTun4BQ81qIRHQ-4pRqiGtraG32wFUfsX5caWUFv8GiXNgBuMiYvI2BNAXEIcOfYigTM7pFhpWe38qM0NSrU03UKrYMsQW7E8tAnUpk_o2Lh99-A==), [ScottsSafetyUK](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUPxaZXax4jhgob0rgI7NOqvBTdw4h9c3r6l7sEwFK_E6Hk5bMYXdinuQKCThfn99rMnQ9C4qg1FEGp4IzbER67XiFBG1cN-QaDABVZ_Plf2DzPjnUTtZNGZ31FKmHun6RNQUzWcqncThrdQ==)
- PFAS management and remediation technologies: Exit PFAS by end of 2025; deployment of GAC, RO, Ion Exchange, Thermal Oxidation; >85% PFAS discharge reduction by end of 2022 vs 2019. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEGnvaPWRix3HotMzOznmOiq2Yxcj3XTOUeDK-scVwV3JY1WqzNhLclIUKhjwTMupgv7zf_WTRV8w5OY3X7HoHvYdc5tkdPl_yLvRBnXm0_5EBsuov6wbbRZyQlLd0tIVd9e516o_HMeyX7MZIlxHi0qWnDS51zB298mkIPsgCC_FdNGpv4i64vAC8=), [WaterOnline](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGZ6esqfI6vmTcn3bhj9l8S0fZsP0hEum0p1zPhzK_E1B7g_dbKyGIKH322m34xQXgDNVZOj4aO7lJKt7AauBcyRTYabUo-7MKXoBaPtay8UVB97niT0cl6jXh0K8-CVYGpAsig38c6drwf3NDO1sTTHkJWicc1ichc9Puwz32xbk_hjx6crlm4K4Wnjz7seq4APO7k8qBWrtkAb3KFSUCUWNJ0D6a8njU=)
- 3M eXcellence and investment in R&D: $3.5B in R&D planned for 2025–2027; 1,000 new products; 6% earnings invested in R&D annually; focus on climate tech, circular economy, and next-gen electronics. [Investing](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFU7KubYCNjI4kAzut1HHe1Y_pUv29VYDWE7IjeQ0zsTiPS1kreWjQ3CS4bx3sI5nTUmHCM5XlU_iw-HruZmHYLVssW47xtt7U6300nNNb_FVJAN64_3UXDPmXiJrmN92EAVxz85FSfaddOP5aa42G1YjMKKTcMb0taZRu1-a9PwKYcGU22E2E8E0Gcjd85YDIvQ-A5IcLUcaXxdViogmYdby7PmvuI2kgF880X52CJAGXwQItOY_MsQTFNH3An3j8KYNpcDXK8Iohz5JJtvw==), [CircularCollective](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF8un6wDql3WuH_ntQiAIt18jycKwdFU5dSLx4dGb27TrD_DSk4eOs8i7aAQjN_8lBUPRX11v4sE2AleM_dS6SfnoPTAKBNU25FbMEas7ln7CINPSEp1SSweA2dALYiMPyKo6t4E7iGhj4pN8nRWSKH0RkEIw5wYerr_93qr_rKSsxz8Tn1SwV6VE7nlyl2), [TriplePundit](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHKnKS9TNCg6g2UPgcYU-wnyw-WWuyelyMrk--cbVHe44EYHTACA99eTUcBl6kkznQg-NnSSp_LAC8_NnFTWrzYUwxco9SqI_pC5aZFY0nFgR0xIhPX6NTM24dHkhAhXBEMPHvZmVxJRRGyH_C_IqzPpgjVM4j04WWt-Jg5B8hYqHf_9vttwJoSTb20pwYZ65ACjiPjJaEwpuvPEVGuYrPLdhvW)
- Industry benchmarking context: Dow and others’ goals and progress; 2024 CDP context; peer comparisons showing 3M’s relative leadership in renewables, SBTi validation, and PFAS actions. [PackagingDive](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHb_K51S1URb74w6x8ScUPxWrtkkWekpcpGfJzy5rq1-pYfdSCdTCD67Fc7u4Plrrz2gWo8Q11JepsEVDBFIttOav9ozGz7gAspct-ZP-Q_u7P0vLeT3zYgpmIs4MgAYcFi572MksDLB8DLISkzUJeW-NJ9pN-_wJDSZJ6fv-or2H1D43lPSaMh8bbRTHLseD6os8BL-bGJSYcSuoL_0mU=), [CDP](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHCbWSmBJq3wh3h69R-uh8HOkJV95rBFLx686B5LFrJLqwuMWSOaPZvItsfH4sbCxxqbh5HsxV52gufx7fkwOCqXHliphwj4AYw6Yn4y1uColPimZ5lwEgrGXRFX38=)

Notes
- All figures above refer to data and targets cited in 2022–2025 sources and 3M public disclosures cited in the provided summaries. For the most granular facility-level data and interim-year breakdowns, consult 3M’s Sustainability Report, Climate Transition Plan (due end of 2025), and Form 10-K filings. [3M](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcrdI7iGrt8DXKMgkjR-IYVPb4DQhYk8tFJ6x4qIQg_iwrNECzIPhx19psPiqkDLxs_-0Pg0JuY27Y9FohRZ_NywetvznKmXU8hq9wh_6WP0H7Ld6L7aTEoX3D-sieUfF-FqaSuvrjYabHUTt_6KfL51h3s-e0-E2RqYAcTPT4CFxzZMVhugen), [Climate Transition Plan](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHcrdI7iGrt8DXKMgkjR-IYVPb4DQhYk8tFJ6x4qIQg_iwrNECzIPhx19psPiqkDLxs_-0Pg0JuY27Y9FohRZ_NywetvznKmXU8hq9wh_6WP0H7Ld6L7aTEoX3D-sieUfF-FqaSuvrjYabHUTt_6KfL51h3s-e0-E2RqYAcTPT4CFxzZMVhugen)